<a href="https://colab.research.google.com/github/HeatherDriver/IU-Model-Engineering/blob/main/01_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install ordered_set
! pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 5.9 MB/s eta 0:00:00


In [2]:
# Relevant imports
import pandas as pd
import numpy as np
import re
from ordered_set import OrderedSet
import datetime as dt
import warnings
import math
from collections import Counter
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import joblib
import os

In [3]:
# Mount Google Drive into the Colab environment and change current directory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive/01_Data/01_Raw'
images_folder = '/content/drive/MyDrive/02_Docs'

/content/drive/MyDrive/01_Data/01_Raw


In [5]:
# Display all columns, rows and import the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

warnings.filterwarnings('ignore')
sns.set(palette="Dark2")

path = '/content/drive/MyDrive/01_Data/01_Raw'

flight_original = pd.read_csv(path +'/flight_information.csv')
ground_original = pd.read_csv(path + '/ground_information.csv')

flight = flight_original.copy(deep=True)
ground = ground_original.copy(deep=True)

**Creating functions for cleaning data, printing null counts**

In [6]:
# Renaming columns to make for easier selection -> Adding underscore between words without changing column order
def add_underscore_to_columns(df):
    cols = []
    for col in df.columns:
        if ' ' in col:
            replacement = str(col).replace(' ', '_')
            df[replacement] = df[col]
            df = df.drop([col], axis=1)
            cols.append(replacement)
        else:
            cols.append(col)
    df = df[cols]
    return df

# Function to cast selected columns to datetime
def to_datetime(list_cols, df):
    for col in list_cols:
        df[col] = pd.to_datetime(df[col])
    return df

# Function to print the count and percentage of missing values by column
def null_printer(df):
    nulls = df.isnull().sum().sort_values(ascending=False)
    perc_null = [i/df.shape[0]*100 for i in nulls]
    null_val = nulls.array
    cols = nulls.index

    print('-' * 80)
    print(f'{"Count & Percentage Missing Value by Column" :^80}')
    print('-' * 80)

    for i, ii, iii in zip(cols, null_val, perc_null):
        print(f"{i:30}: {ii:<15}:{iii:30}")

# Simplifying departure date with day_of_origin additional column
def creates_day_of_origin(df, column_to_use):
    df['day_of_origin'] = pd.to_datetime(df[column_to_use], format='%Y-%m-%d')
    # Extracting only the date component
    df['day_of_origin'] = df['day_of_origin'].dt.date
    return df

# Add the day of week of the flight, where 0 : Monday and 6: Sunday.
# This is to help with a stratified sample, since day_of_origin is too granular.
def creates_day_of_week(df):
    df['day_of_origin'] = pd.to_datetime(df['day_of_origin'])
    df['day_of_week'] = df['day_of_origin'].dt.dayofweek.astype('str')
    df['day_of_origin'] = df['day_of_origin'].dt.date
    return df

# creates duration calculation, this is the difference in time for the arrival versus the departure.
def creates_duration_col(df, start_col, end_col):
    df2 = df.copy(deep=True)
    if start_col == 'dep_sched_time' :
        # Convert 'dep_sched_date' and 'arr_sched_date' to datetime if not already
        df2[start_col] = pd.to_datetime(df2['dep_sched_date'].astype('str') + ' ' + df2['dep_sched_time'].astype('str'), format="%Y-%m-%d %H:%M")
    if end_col == 'arr_sched_time':
        df2[end_col] = pd.to_datetime(df2['arr_sched_date'].astype('str') + ' ' + df2['arr_sched_time'].astype('str'), format="%Y-%m-%d %H:%M")

    if df2[start_col].dtype != 'datetime64[ns]':
        df2[start_col] = pd.to_datetime(df2[start_col])
    if df2[end_col].dtype != 'datetime64[ns]':
        df2[end_col] = pd.to_datetime(df2[end_col])

    duration = (df2[end_col] -  df2[start_col])/ np.timedelta64(1, 'm')
    return duration.values

**Functions for visualising univariates**

In [7]:
# Freedman-Diaconis rule used to calculate the number of bins in the histogram. It calculates the interquartile range and divides this by a scaling factor.
# The scaling factor = 2 x cube root of the count of data points.

def freedman_diaconis(df, col_name):
    data = df[col_name].dropna()
    iqr_manual = np.quantile(data, q=[.25, .75])
    iqr = np.diff(iqr_manual)[0]
    N = data.shape[0]
    bw   = (2 * iqr) / np.power(N, 1/3)
    datmin, datmax = data.min(), data.max()
    datrng = datmax - datmin
    result = int((datrng / bw) + 1)
    return result

# Non-null values analysed within univariate analyses (boxplot and histograms) - fn plots these
def boxplot_hist_plotter(df, col_name, image_name=False):
    title = col_name + ' data analysis'
    colours = np.random.randint(len(px.colors.qualitative.Bold), size=2)
    _for_boxplot = df.loc[df[col_name].notnull(), [col_name]].values
    _for_boxplot = [value for v in _for_boxplot for value in v]

    # Freedman-Diaconis to calculate the number of bins
    fd = freedman_diaconis(df, col_name)
    # Plots boxplot and histogram
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Boxplot", "Histogram"))
    fig.add_trace(
        go.Box(y=_for_boxplot, name= '', marker=dict(color=px.colors.qualitative.Bold[colours[0]])),
        row=1, col=1
    )

    fig.add_trace(
        go.Histogram(x=df[col_name], histnorm='probability', nbinsy=fd, marker=dict(color=px.colors.qualitative.Bold[colours[1]])),
        row=1, col=2
    )
    fig.update_layout(height=400, width=1000, title_text=title, showlegend=False)
    if image_name:
      fig.update_layout(title=None)
      fig.write_image(images_folder + "/" + image_name)
    fig.show()

# Calculates basic statistics of the numeric columns with the mode
def col_statistics(df, col_name, print_skew=True):
    x = df[[col_name]].describe()
    y = df.loc[df[col_name].notnull(), [col_name]].mode().iloc[[0]]
    y.index = ['mode']
    z = df.loc[df[col_name].notnull(), [col_name]].median()
    z.index = ['median']
    a = pd.DataFrame(z, columns=[col_name])
    summary = pd.concat([x, y, a], axis = 0)
    if print_skew:
    # Right skewed = mode < median < mean.
        if summary.loc['mode'].values[0] < summary.loc['median'].values[0] < summary.loc['mean'].values[0]:
            print(col_name + ' is right skewed (positively skewed)')
     # Left skewed = mean < median < mode.
        if summary.loc['mean'].values[0] < summary.loc['median'].values[0] < summary.loc['mode'].values[0]:
            print(col_name + ' is left skewed (negatively skewed)')

    return summary

# For blank cells (Sched_Groundtime and Act_Groundtime), plots the incidence of these and the time difference to the next line.
# This is to confirm that these blanks exist for the last flight of each day, per ac_registration.
def plots_duration_after_blank_col(col_name):
    ac_registration = OrderedSet(flight['ac_registration'].values)

    mylisti, mylistii = [], []
    for reg in ac_registration:
        investigation = flight.loc[(flight[col_name].isna()) & (flight['ac_registration'] == reg)]
        idx = investigation.index.values[:-1]

        idx_plus_one = idx + 1
        investigation = flight.iloc[idx]
        investigation_next_day = flight.iloc[idx_plus_one]
        investigation = investigation.reset_index()
        investigation_next_day = investigation_next_day.reset_index()

        investigation['day_diff'] = investigation_next_day['day_of_origin'] - investigation['day_of_origin']
        perc_summary = investigation['day_diff'].value_counts()
        mylisti.append(perc_summary.index.array)
        mylistii.append(perc_summary.values)

    mylistii = [v for val in mylistii for v in val]
    mylisti = [v for val in mylisti for v in val]

    to_plot = pd.DataFrame(mylistii, index=mylisti)
    to_plot.columns = ['Count']
    to_plot = to_plot.reset_index()

    # count of days lapsing after a blank value for col_name
    to_plot = pd.pivot_table(to_plot, index='index', aggfunc=sum)
    to_plot = to_plot.reset_index()
    to_plot['Time_difference'] = to_plot['index'].astype('str')

    fig = px.bar(to_plot, y='Time_difference', color='Time_difference', x='Count', orientation='h',
                 height=400, color_discrete_sequence=px.colors.qualitative.Bold,
                 title='Time difference between blank cells and row below for ' + col_name)

    fig.update_layout(showlegend=False)
    fig.show()

# 1. Data examination and deduplication

## Flight file

**Supplied flight information column definitions**
1. <span style="background-color: #8DB0CE">Unnamed: 0:</span> Index
2. <span style="background-color: #8DB0CE">leg_no:</span> Unique identifier of a flight on a given day, at a certain time, with a given flight number
3. <span style="background-color: #8DB0CE">fn_carrier:</span> Airline name
4. <span style="background-color: #8DB0CE">fn_number:</span> Flight number; has to be unique on a given day
5. <span style="background-color: #8DB0CE">dep_ap_sched:</span> Scheduled departure airport
6. <span style="background-color: #8DB0CE">arr_ap_sched:</span> Scheduled arrival airport
7. <span style="background-color: #8DB0CE">dep_sched_date:</span> Scheduled departure date
8. <span style="background-color: #8DB0CE">dep_sched_time:</span> Scheduled departure time
9. <span style="background-color: #8DB0CE">arr_sched_date:</span>  Scheduled arrival date
10. <span style="background-color: #8DB0CE">arr_sched_time:</span>  Scheduled arrival time
11. <span style="background-color: #8DB0CE">m_offblockdt:</span>  Timestamp of departure
12. <span style="background-color: #8DB0CE">m_onblockdt:</span> Timestamp of arrival
13. <span style="background-color: #8DB0CE">ac_registration:</span> Aircraft registration number, i.e. the “license plate” of the aircraft
14. <span style="background-color: #8DB0CE">change_reason_code:</span> Reason for delay (assigned after the flight)
15. <span style="background-color: #8DB0CE">dep_delay:</span> Departure delay
16. <span style="background-color: #8DB0CE">Ac Type Code:</span> Aircraft type (example: 320 = Airbus A320)
17. <span style="background-color: #8DB0CE">trans_time:</span> True minimal transition time for crew members after flight, i.e. transition time of the crew
member with the least time
18. <span style="background-color: #8DB0CE">sched_trans_time:</span> Scheduled minimal transition time for crew members after flight, i.e. scheduled transition time of the crew member with the least time
19. <span style="background-color: #8DB0CE">Crew Group:</span> Assignment of what happens to the whole crew after a flight \
o Start : First flight of day \
o A : all crew members stay on the aircraft for the next flight \
o B, B2: all crew members switch aircraft for the next flight \
o C: at least one crew member switches aircraft for the next flight
20. <span style="background-color: #8DB0CE">TLC_trans:</span> Names of crew members on flight with some additional information attached to each name
(but business does not know exactly which additional information is visible there)
21. <span style="background-color: #8DB0CE">crew_type_change:</span> Rank (cp = pilot, ca = cabin member) of crew members, who changed aircraft
22. <span style="background-color: #8DB0CE">Sched Groundtime:</span> Scheduled ground time of the aircraft between flights
23. <span style="background-color: #8DB0CE">Act Groundtime:</span> Actual ground time of the aircraft between flights.
24. <span style="background-color: #7CFC00">day_of_origin:</span> (Imputed) Day of flight departure.
25. <span style="background-color: #7CFC00">day_of_week:</span> (Imputed) Weekday of flight departure where 0: Monday and 6: Sunday.
26. <span style="background-color: #7CFC00">sched_duration:</span> (Imputed) Difference between arr_sched_time and dep_sched_time.
27. <span style="background-color: #7CFC00">actual_duration:</span> (Imputed) Difference between m_onblockdt and m_offblockdt.

In [8]:
# Add imputed columns to Flight
flight = creates_day_of_origin(flight, 'dep_sched_date')
flight = creates_day_of_week(flight)

flight['sched_duration'] = creates_duration_col(flight, 'dep_sched_time', 'arr_sched_time')
flight['sched_duration'] = np.round(flight['sched_duration'], 0)

flight['actual_duration'] = creates_duration_col(flight, 'm_offblockdt', 'm_onblockdt')
flight['actual_duration'] = np.round(flight['actual_duration'], 0)

In [9]:
# Check dtypes as loaded so can cast these to another type if required.
flight.dtypes

Unnamed: 0              int64
leg_no                  int64
fn_carrier             object
fn_number              object
dep_ap_sched           object
arr_ap_sched           object
dep_sched_date         object
dep_sched_time         object
arr_sched_date         object
arr_sched_time         object
m_offblockdt           object
m_onblockdt            object
ac_registration        object
change_reason_code     object
dep_delay             float64
Ac Type Code           object
trans_time              int64
sched_trans_time        int64
Crew Group             object
TLC_trans              object
crew_type_change       object
Sched Groundtime      float64
Act Groundtime        float64
day_of_origin          object
day_of_week            object
sched_duration        float64
actual_duration       float64
dtype: object

## Ground file

**Supplied ground information column definitions:**
1. <span style="background-color: #FFA833">Unnamed:_0:</span> Index
2. <span style="background-color: #FFA833">day_of_origin:</span> day of flight (also given in Flight Information)
3. <span style="background-color: #FFA833">ac_type:</span> aircraft type (also given in Flight Information)
4. <span style="background-color: #FFA833">fn_number:</span> flight number (also given in Flight Information)
5. <span style="background-color: #FFA833">ac_registration:</span> aircraft registration (also given in Flight Information)
6. <span style="background-color: #FFA833">mingt:</span> minimal scheduled ground time for the given aircraft
7. <span style="background-color: #FFA833">dep_leg_inbound:</span> departure airport name of inbound (=arriving) flight
8. <span style="background-color: #FFA833">arr_leg_inbound:</span> arrival airport name of inbound (=arriving) flight
9. <span style="background-color: #FFA833">arr_leg_outbound:</span> departure airport name of outbound (=departing) flight
10. <span style="background-color: #FFA833">sched_inbound_dep:</span> scheduled departure time of inbound (=arriving) flight
11. <span style="background-color: #FFA833">sched_inbound_arr:</span> scheduled arrival time of inbound (=arriving) flight
12. <span style="background-color: #FFA833">sched_outbound_dep:</span> scheduled departure time of outbound (=departing) flight
13. <span style="background-color: #FFA833">sched_outbound_arr:</span> scheduled arrival time of outbound (=departing) flight
14. <span style="background-color: #FFA833">sched_turnaround:</span> scheduled ground time for aircraft
15. <span style="background-color: #FFA833">leg_inbound:</span> leg number of inbound flight
16. <span style="background-color: #FFA833">leg_outbound:</span> leg number of outbound flight
17. <span style="background-color: #FFA833">catering_duration:</span> catering duration (i.e. filling up meal boxes) between flights in minutes
18. <span style="background-color: #FFA833">cleaning_duration:</span> cleaning duration between flights in minutes
19. <span style="background-color: #FFA833">pax_boarding_duration:</span> boarding duration between flights in minutes

In [10]:
# Check dtypes as loaded so can cast these to another type if required.
ground.dtypes

Unnamed: 0                 int64
day_of_origin             object
ac_type                   object
fn_number                 object
ac_registration           object
mingt                      int64
dep_leg_inbound           object
arr_leg_inbound           object
arr_leg_outbound          object
sched_inbound_dep         object
sched_inbound_arr         object
sched_outbound_dep        object
sched_outbound_arr        object
sched_turnaround           int64
leg_inbound                int64
leg_outbound               int64
catering_duration          int64
cleaning_duration        float64
pax_boarding_duration    float64
dtype: object

## Basic preprocessing to prepare for splitting

In [11]:
# Recast leg_no as string, not an integer.
flight['leg_no'] = flight['leg_no'].astype('str')

# dep_sched_date and dep_sched_time will be combined to a datetime column (same as arr_sched_date and arr_sched_time).
# m_offblockdt and m_onblockdt will be cast to a datetime column.
flight['dep_sched_datetime'] = flight['dep_sched_date'].astype(str) + ' ' + flight['dep_sched_time'].astype(str)
flight['arr_sched_datetime'] = flight['arr_sched_date'].astype(str) + ' ' + flight['arr_sched_time'].astype(str)
to_datetime_list = ['m_offblockdt', 'm_onblockdt', 'dep_sched_datetime', 'arr_sched_datetime']
flight = to_datetime(to_datetime_list, flight)

# Add underscores for ease of use
flight = add_underscore_to_columns(flight)

# Change 'dep_delay' to be an integer of minutes
flight['dep_delay'] = flight['dep_delay'].astype('int')

# Drop 'Unnamed: 0' column because is replicated in the index
flight.drop(['Unnamed:_0'], axis=1, inplace=True)

In [12]:
# Convert datatypes of columns if required and add underscores
ground = add_underscore_to_columns(ground)

# leg_inbound, leg_outbound should be strings
ground['leg_inbound'] = ground['leg_inbound'].astype(str)
ground['leg_outbound'] = ground['leg_outbound'].astype(str)

# Drop 'Unnamed:_0' column because is replicated in the index
ground.drop(['Unnamed:_0'], axis=1, inplace=True)

# Add day of the week
ground = creates_day_of_week(ground)

### Train and test dataset rationale

In [13]:
# Analysis of the task shows that prediction of the following variables is needed:
# 1. dep_delay: this is because dep_sched_time + dep_delay = m_offblockdt
# 2. flight_duration_actual: m_offblockdt - m_onblockdt. Thus predicting this can be used to calculate back the m_onblockdt.
# 3. flight_duration_sched: arr_sched_time - dep_sched_time. Simularly predicting this can be used to calculate back the arr_sched_time.
# 4. Sched_Groundtime: Adding Sched_Groundtime and arr_sched_time = dep_sched_time for the following flight (dep_sched_time + 1).
# 5. Act_Groundtime: Act_Groundtime + m_onblockdt = m_offblockdt for the following flight (m_offblockdt + 1)

In [14]:
# Need to understand the gaps in the data between the 2 files - eg what registrations are represented in one and not the other, even additional flights not represented.
_flight_group = flight[['ac_registration', 'day_of_origin', 'day_of_week', flight.columns[0]]].groupby(
    by=['ac_registration', 'day_of_week', 'day_of_origin']).count().reset_index(drop=False)
_flight_group.columns = ['ac_registration', 'day_of_origin', 'day_of_week', 'count']

_ground_group = ground[['ac_registration', 'day_of_origin', 'day_of_week', ground.columns[1]]].groupby(
    by=['ac_registration', 'day_of_week', 'day_of_origin']).count().reset_index(drop=False)
_ground_group.columns = ['ac_registration', 'day_of_origin', 'day_of_week', 'count']

merged_group = _flight_group.merge(_ground_group, how='outer', left_on=['ac_registration', 'day_of_origin', 'day_of_week'],
                    right_on=['ac_registration','day_of_origin', 'day_of_week'],
                    suffixes=('_flight', '_ground'))

# View gaps in the data from the combined summary
flight_gaps = merged_group[merged_group['count_flight'].isna()][['ac_registration']].value_counts()
print('flight gaps:\n', flight_gaps)
ground_gaps = merged_group[merged_group['count_ground'].isna()]['ac_registration'].value_counts()
print('\nground gaps:\n', ground_gaps.head())

flight gaps:
 Series([], Name: count, dtype: int64)

ground gaps:
 ac_registration
ECLGKX    17
ECLGGX    15
ECLGCX    12
ECLWLX    10
ECLWOX    10
Name: count, dtype: int64


In [15]:
# Need to find a sufficiently represented sample to split the data to train and test sets.
for_splitting = _flight_group[['ac_registration', 'day_of_origin', 'day_of_week']].groupby(by=['ac_registration', 'day_of_origin']).count().reset_index(drop=False)
for_splitting.columns = ['ac_registration', 'day_of_origin', 'count']
for_splitting['day_of_origin'] = for_splitting['day_of_origin'].astype('str')
for_splitting['count'].value_counts()

count
4    320
5    126
3     31
1      8
2      1
Name: count, dtype: int64

In [16]:
# While flight is the better represented file, the stratified train test split cannot be constructed with ac_registration as it stands, due to low
# representation of some aircraft registrations (see count of 1s above).
# Thus a quantile classification of the total flight count per ac_registration will be created as a feature, this will then mean the stratified
# train test selector can be created. This will then determine the date and quantile combinations to base the allocation of the train set
# and the test set.

# It makes sense to first remove null, blanks and duplicates prior to spltting into the train and test datasets.

### Null and blank cell investigation

In [17]:
flight = flight.sort_values(['ac_registration', 'dep_sched_datetime'], ascending = [True, True]).reset_index(drop=True)

In [18]:
# Evaluate nulls
null_printer(flight)

--------------------------------------------------------------------------------
                   Count & Percentage Missing Value by Column                   
--------------------------------------------------------------------------------
Act_Groundtime                : 2075           :             17.26720479320962
Sched_Groundtime              : 2013           :            16.751269035532996
actual_duration               : 83             :            0.6906881917283848
m_onblockdt                   : 83             :            0.6906881917283848
leg_no                        : 0              :                           0.0
trans_time                    : 0              :                           0.0
dep_sched_datetime            : 0              :                           0.0
sched_duration                : 0              :                           0.0
day_of_week                   : 0              :                           0.0
day_of_origin                 : 0             

In [19]:
# Blanks for Act_Groundtime, Sched_Groundtime evaluated
mylist_i, mylist_ii, mylist_iii = [], [], []
for ac_reg in OrderedSet(flight['ac_registration']):
    for origin_date in OrderedSet(flight['day_of_origin']):
        subset = flight.loc[(flight['ac_registration'] == ac_reg) & (flight['day_of_origin'] == origin_date)]['dep_sched_datetime']
        mylist_i.append(subset.rank(ascending=False).values)
        mylist_ii.append(subset.rank(ascending=True).values)
        mylist_iii.append(subset.index.values)

mylist_i = [v for values in mylist_i for v in values]
mylist_ii = [v for values in mylist_ii for v in values]
mylist_iii = [v for values in mylist_iii for v in values]

mydict_i = {idx: val for (idx, val) in zip(mylist_iii, mylist_i)}
mydict_ii = {idx: val for (idx, val) in zip(mylist_iii, mylist_ii)}

# Blanks for last and first flight evaluated
flight['last_flight_indicator'] = flight.index.map(mydict_i)
flight['last_flight_indicator'] = np.where(flight['last_flight_indicator'] == 1, 1, 0)
flight['first_flight_indicator'] = flight.index.map(mydict_ii)
flight['first_flight_indicator'] = np.where(flight['first_flight_indicator'] == 1, 1, 0)

flight['last_and_blank_act_gt'] = 0
flight.loc[(flight['last_flight_indicator'] == 1) & (flight['Act_Groundtime'].isna()), ['last_and_blank_act_gt']] = 1

flight['first_and_blank_act_gt'] = 0
flight.loc[(flight['first_flight_indicator'] == 1) & (flight['Act_Groundtime'].isna()), ['first_and_blank_act_gt']] = 1

flight['last_and_blank_sched_gt'] = 0
flight.loc[(flight['last_flight_indicator'] == 1) & (flight['Sched_Groundtime'].isna()), ['last_and_blank_sched_gt']] = 1

flight['first_and_blank_sched_gt'] = 0
flight.loc[(flight['first_flight_indicator'] == 1) & (flight['Sched_Groundtime'].isna()), ['first_and_blank_sched_gt']] = 1

In [20]:
# Visualising the results
investigation_summary = pd.DataFrame(columns=['Sched_Groundtime', 'Act_Groundtime'], index=['blank count', 'first flight', 'last flight', 'other'])

investigation_summary.loc['blank count']['Sched_Groundtime'] = flight['Sched_Groundtime'].isna().sum()
investigation_summary.loc['first flight']['Sched_Groundtime'] = flight['first_and_blank_sched_gt'].sum()
investigation_summary.loc['last flight']['Sched_Groundtime'] = flight['last_and_blank_sched_gt'].sum()
investigation_summary.loc['other']['Sched_Groundtime'] = investigation_summary.loc['blank count']['Sched_Groundtime'] - investigation_summary.loc[
'first flight']['Sched_Groundtime'] - investigation_summary.loc['last flight']['Sched_Groundtime']

investigation_summary.loc['blank count']['Act_Groundtime'] = flight['Act_Groundtime'].isna().sum()
investigation_summary.loc['first flight']['Act_Groundtime'] = flight['first_and_blank_act_gt'].sum()
investigation_summary.loc['last flight']['Act_Groundtime'] = flight['last_and_blank_act_gt'].sum()
investigation_summary.loc['other']['Act_Groundtime'] = investigation_summary.loc['blank count']['Act_Groundtime'] - investigation_summary.loc[
'first flight']['Act_Groundtime'] - investigation_summary.loc['last flight']['Act_Groundtime']

investigation_summary = investigation_summary.reset_index(drop=False)
investigation_summary = pd.melt(investigation_summary, id_vars=['index'], value_vars=['Sched_Groundtime', 'Act_Groundtime'])
investigation_summary.rename(columns={'index':'indicator'}, inplace=True)
investigation_summary = investigation_summary.sort_values(['variable', 'value'], ascending = [True, False]).reset_index(drop=True)

fig = px.bar(investigation_summary, x="indicator", y="value", color="indicator", facet_col="variable", height=500,
             color_discrete_sequence=px.colors.qualitative.Bold, title='Blank cells for Act_Groundtime & Sched_Groundtime versus blank first and last flight indicators')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_yaxes(matches=None)
fig.update_layout(title=None)
fig.write_image(images_folder + "/blanks.png")
fig.show()

In [21]:
# This shows that where Act_Groundtime & Sched_Groundtime is blank and the last_flight_indicator is 1, a different impution strategy will be
# used. These values can be replaced by 0, because the large value that will be imputed using the next day will negatively affect the model.

# Where Act_Groundtime & Sched_Groundtime is blank and the first_flight_indicator is 1, these values will be imputed through a lagged calculation.

# Need to add a flight count per day as well, since some ac_reg have only 1 flight a day, so last_flight_indicator and first_flight_indicator are
# both 1.

# However data cleansing and row deduplication is needed before imputing so that the calculations can be accurate - this is since these
# run on a lagged basis with the row below.

### Flight data cleansing: row deduplication

In [22]:
# Removing duplicate rows prior to running lagged calculations is needed for accuracy.
# Similarly the removal of duplicate rows is required before splitting into a training and test set.

In [23]:
# Creating function to check duplicates
def flight_finds_duplicates(df, cols):
    _df = df.duplicated(subset=cols, keep=False)
    df['duplicate_row'] = _df
    df = df.sort_values(['ac_registration', 'dep_sched_datetime'], ascending = [True, True]).reset_index(drop=True)
    return df

In [24]:
# Duplicates existing on registration plate number, flight_train_df number, date, time
flight = flight_finds_duplicates(flight, ['ac_registration', 'fn_number', 'dep_sched_datetime', 'arr_sched_datetime'])
investigation = flight.loc[flight['duplicate_row'] == True]
investigation.head(2)

,leg_no,fn_carrier,fn_number,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration,change_reason_code,dep_delay,Ac_Type_Code,trans_time,sched_trans_time,Crew_Group,TLC_trans,crew_type_change,Sched_Groundtime,Act_Groundtime,day_of_origin,day_of_week,sched_duration,actual_duration,dep_sched_datetime,arr_sched_datetime,last_flight_indicator,first_flight_indicator,last_and_blank_act_gt,first_and_blank_act_gt,last_and_blank_sched_gt,first_and_blank_sched_gt,duplicate_row
1222,272218014,East Carmen Airlines,EC11525,East Allisontown,Aaronbury,2019-06-29,16:25,2019-06-29,17:05,2019-06-29 16:27:00,2019-06-29 17:07:00,ECLBJX,other problem,2,320,33,45,A,"['Erika Sandoval_232_33_45_cp', 'Julia Hollowa...",[],-40.0,64.0,2019-06-29,5,40.0,40.0,2019-06-29 16:25:00,2019-06-29 17:05:00,0,0,0,0,0,0,True
1223,272218014,East Carmen Airlines,EC11525,East Allisontown,Aaronbury,2019-06-29,16:25,2019-06-29,17:05,2019-06-29 16:28:00,2019-06-29 17:08:00,ECLBJX,other problem,3,320,0,0,Start,"['Robert Williams_nan_nan_nan_ca', 'Holly Cook...",[],50.0,50.0,2019-06-29,5,40.0,40.0,2019-06-29 16:25:00,2019-06-29 17:05:00,0,0,0,0,0,0,True


In [25]:
# Need to understand on what basis the duplicate rows differ from each other so can judge which of the duplicate is closer
# to a feasible situation and remove the other row.
def sort_dict_by_value(d, reverse = False):
    return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))

def prints_differing_cols(df, indices=None):
    if 'index' not in df.columns:
        df = df.reset_index()
    dup_list = df['index'].to_list()
    dup_list_odd = dup_list.copy()
    dup_list_odd = dup_list_odd[::2]

    dup_list_eve = dup_list.copy()
    dup_list_eve = dup_list_eve[1::2]

    # Want to check on what columns the duplicate rows actually do differ on so can prove why should choose one row over the other (i.e
    # looking for high cardinality columns within the duplicates)
    mylist = []
    for i, ii in zip(dup_list_odd, dup_list_eve):
        selector = df.loc[(df['index'] == i) | (df['index'] == ii)]
        cols = list(selector.columns)
        for col in cols:
            if col in ['index', 'leg_no', 'fn_carrier', 'fn_number', 'ac_registration', 'reg_flightnum_dep_arr_dup', 'dup_num', 'TLC_trans',
                       'dep_ap_sched', 'arr_ap_sched', 'level_0']:
                continue
            else:
                check_same = selector.duplicated(subset=[col], keep=False)
                if not check_same.values.any():
                    mylist.append(col)

    count_dict = Counter(mylist)
    count_dict = {key: value*2 for key, value in zip(count_dict.keys(), count_dict.values())}
    if indices:
        return (dup_list_odd, dup_list_eve)
    else:
        return sort_dict_by_value(count_dict, reverse=True)

In [26]:
to_plot_ = prints_differing_cols(investigation)
to_plot = {'column name':[keys for keys in to_plot_.keys()], 'cardinality number':[values for values in to_plot_.values()]}

In [27]:
fig = px.bar(to_plot, x="cardinality number", y="column name", color='cardinality number', orientation='h',
             text='cardinality number', height=500, color_continuous_scale=px.colors.sequential.Magenta,
             title='Number of unique values per column for duplicate data')

fig.update_layout(showlegend=False,)
fig.update_coloraxes(showscale=False)
fig.show()

In [28]:
# Drop the duplicate rows where Sched_Groundtime or Act_Groundtime is negative since a negative time seems implausible.
dup_list_odd = prints_differing_cols(investigation, 'y')[0]
dup_list_eve = prints_differing_cols(investigation, 'y')[1]

investigation = investigation.reset_index()

mylist = []
for odd, even in zip(dup_list_odd, dup_list_eve):
    subset = investigation.loc[(investigation['index'] == odd) | (investigation['index'] == even)]
    subset['Sched_Groundtime_neg'] = subset['Sched_Groundtime'].where(subset['Sched_Groundtime'] > 0, 'drop')
    summary = subset['Sched_Groundtime_neg'].value_counts()
    try:
        idx_finder = summary.loc['drop']
        if idx_finder == 1:
            idx = subset.loc[subset['Sched_Groundtime_neg'] == 'drop']['index']
            mylist.append(idx.values[0])
    except:
        continue

mylist_ = []
for odd, even in zip(dup_list_odd, dup_list_eve):
    subset = investigation.loc[(investigation['index'] == odd) | (investigation['index'] == even)]
    subset['Act_Groundtime_neg'] = subset['Act_Groundtime'].where(subset['Act_Groundtime'] > 0, 'drop')
    summary = subset['Act_Groundtime_neg'].value_counts()
    try:
        idx_finder = summary.loc['drop']
        if idx_finder == 1:
            idx = subset.loc[subset['Act_Groundtime_neg'] == 'drop']['index']
            mylist_.append(idx.values[0])
    except:
        continue

mylist.extend([idx for idx in mylist_ if idx not in mylist])

flight.drop(mylist, inplace=True)

In [29]:
# Refind duplicates existing on registration plate number, flight number, date, time
flight = flight_finds_duplicates(flight, ['ac_registration', 'fn_number', 'dep_sched_datetime', 'arr_sched_datetime'])
investigation = flight.loc[flight['duplicate_row'] == True]
investigation.shape

(34, 35)

In [30]:
prints_differing_cols(investigation)

{'trans_time': 34,
 'Act_Groundtime': 32,
 'sched_trans_time': 22,
 'm_offblockdt': 20,
 'dep_delay': 20,
 'Sched_Groundtime': 20,
 'm_onblockdt': 18,
 'change_reason_code': 18,
 'Crew_Group': 12}

In [31]:
# Since duplicates differ on the basis of trans_time, can analyse the distribution of trans_time
_for_boxplot = flight[['trans_time']]
boxplot_hist_plotter(_for_boxplot, 'trans_time')

In [32]:
col_statistics(_for_boxplot, 'trans_time')

trans_time is right skewed (positively skewed)


,trans_time
count,11994.000000
mean,27.710772
std,31.906935
min,-134.000000
25%,0.000000
50%,27.000000
75%,49.000000
max,470.000000
mode,0.000000
median,27.000000


In [33]:
# Can drop the duplicate rows where trans_time is not zero and the duplicate row is zero.
# this is due to the class imbalance in the data that does indicate that zero is a commonly occurring value.
dup_list_odd = prints_differing_cols(investigation, 'y')[0]
dup_list_eve = prints_differing_cols(investigation, 'y')[1]

investigation = investigation.reset_index()

mylist = []
for odd, even in zip(dup_list_odd, dup_list_eve):
    subset = investigation.loc[(investigation['index'] == odd) | (investigation['index'] == even)]
    subset['trans_time_zero'] = subset['trans_time'].where(subset['trans_time'] == 0, 'drop')
    summary = subset['trans_time_zero'].value_counts()
    try:
        idx_finder = summary.loc['drop']
        if idx_finder == 1:
            idx = subset.loc[subset['trans_time_zero'] == 'drop']['index']
            mylist.append(idx.values[0])
    except:
        continue

flight.drop(mylist, inplace=True)

In [34]:
# Refind duplicates existing on registration plate number, flight number, date, time
flight = flight_finds_duplicates(flight, ['ac_registration', 'fn_number', 'dep_sched_datetime', 'arr_sched_datetime'])
investigation = flight.loc[flight['duplicate_row'] == True]
investigation.shape

(22, 35)

In [35]:
prints_differing_cols(investigation)

{'trans_time': 22,
 'Act_Groundtime': 20,
 'Sched_Groundtime': 12,
 'm_offblockdt': 10,
 'dep_delay': 10,
 'sched_trans_time': 10,
 'm_onblockdt': 8,
 'change_reason_code': 8}

In [36]:
# Remaining duplicates will be deduped on the basis of what the most commonly occurring value is for non-duplicate rows,
# for TLC_trans_1 etc on the same day of origin, for the ac_registration.

# Analyse the distribution of Act_Groundtime_imputed
_for_boxplot = flight[['Act_Groundtime']]
boxplot_hist_plotter(_for_boxplot, 'Act_Groundtime', 'Act_Groundtime.png')

In [37]:
col_statistics(_for_boxplot, 'Act_Groundtime')

,Act_Groundtime
count,9914.000000
mean,66.999899
std,69.875642
min,-383.000000
25%,34.000000
50%,60.000000
75%,87.000000
max,947.000000
mode,67.000000
median,60.000000


In [38]:
# Due to the normal distribution of the data we can favour the rows where the Act_Groundtime is closer to the most probabalistic
# outcome for the aircraft registration.

In [39]:
# Remaining duplicates will be deduped on the basis of what the most commonly occurring value is for non-duplicate rows,
# for Sched_Groundtime etc, for the ac_registration.

def dedupe_on_col_mode(col_name):
    ac_registration = OrderedSet(investigation['ac_registration'].values)

    mylist = []
    for reg in ac_registration:
        subset = flight.loc[(flight['ac_registration'] == reg) & (flight['duplicate_row'] == False)]
        subset_summary = subset[col_name].mode()
        mylist.append((reg, subset_summary.values[0]))

    dup_list_odd = prints_differing_cols(investigation, 'y')[0]
    dup_list_eve = prints_differing_cols(investigation, 'y')[1]

    to_not_drop = []
    for reg, mode in mylist:
        mode_lower = mode - 50
        mode_upper = mode + 50

        target = investigation.loc[(investigation['ac_registration'] == reg) & (investigation[col_name].notna())]
        target[col_name].astype('int')

        subset = target.loc[investigation[col_name] == mode]
        if subset.empty:
            subset = investigation.loc[(investigation['ac_registration'] == reg) & ((investigation[col_name] >= mode_lower) &
                                                                                    (investigation[col_name] <= mode_upper))]
        if not subset.empty:
            to_not_drop.append(subset.index[0])

    to_drop = []
    for odd, even in zip(dup_list_odd, dup_list_eve):
        if odd in to_not_drop:
            to_drop.append(even)
        if even in to_not_drop:
            to_drop.append(odd)

    flight.drop(to_drop, inplace=True)
    return flight

In [40]:
flight = dedupe_on_col_mode('Act_Groundtime')
flight.shape

(11986, 35)

In [41]:
# Refind duplicates existing on registration plate number, flight number, date, time
flight = flight_finds_duplicates(flight, ['ac_registration', 'fn_number', 'dep_sched_datetime', 'arr_sched_datetime'])
investigation = flight.loc[flight['duplicate_row'] == True]
investigation.shape

(18, 35)

In [42]:
prints_differing_cols(investigation)

{'trans_time': 18,
 'Act_Groundtime': 16,
 'Sched_Groundtime': 8,
 'm_offblockdt': 6,
 'dep_delay': 6,
 'change_reason_code': 6,
 'sched_trans_time': 6,
 'm_onblockdt': 4}

In [43]:
# Remaining duplicates will be deduped on the basis of what value is closer to the IQR for Act_Groundtime, since this also has high cardinality.

ac_registration = OrderedSet(investigation['ac_registration'].values)
iqr = col_statistics(_for_boxplot, 'Act_Groundtime')
iqr = iqr.loc[['25%', '75%']].values
iqr = [val[0] for val in iqr]

dup_list_odd = prints_differing_cols(investigation, 'y')[0]
dup_list_eve = prints_differing_cols(investigation, 'y')[1]

to_drop = []
for odd, even in zip(dup_list_odd, dup_list_eve):
    o_subset = flight.iloc[[odd]]['Act_Groundtime'].values[0]
    e_subset = flight.iloc[[even]]['Act_Groundtime'].values[0]
    if o_subset + iqr[0] > e_subset + iqr[0] and o_subset + iqr[1] > e_subset + iqr[1]:
        to_drop.append(even)
    if e_subset + iqr[0] > o_subset + iqr[0] and e_subset + iqr[1] > o_subset + iqr[1]:
        to_drop.append(odd)

flight.drop(to_drop, inplace=True)

In [44]:
# Refind duplicates existing on registration plate number, flight number, date, time
flight = flight_finds_duplicates(flight, ['ac_registration', 'fn_number', 'dep_sched_datetime', 'arr_sched_datetime'])
investigation = flight.loc[flight['duplicate_row'] == True]
investigation.shape

(2, 35)

In [45]:
# Check the data for the final remaining duplicate
investigation

,leg_no,fn_carrier,fn_number,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration,change_reason_code,dep_delay,Ac_Type_Code,trans_time,sched_trans_time,Crew_Group,TLC_trans,crew_type_change,Sched_Groundtime,Act_Groundtime,day_of_origin,day_of_week,sched_duration,actual_duration,dep_sched_datetime,arr_sched_datetime,last_flight_indicator,first_flight_indicator,last_and_blank_act_gt,first_and_blank_act_gt,last_and_blank_sched_gt,first_and_blank_sched_gt,duplicate_row
10936,272088457,East Carmen Airlines,EC11870,Jamesview,South Victoria,2019-06-09,07:00,2019-06-09,07:30,2019-06-09 07:01:00,NaT,ECLWQX,other problem,1,E95,40,40,A,"['Nathan Sharp_23_40_40_cp', 'Elizabeth Garcia...",[],45.0,NaN,2019-06-09,6,30.0,NaN,2019-06-09 07:00:00,2019-06-09 07:30:00,0,0,0,0,0,0,True
10937,272088457,East Carmen Airlines,EC11870,Jamesview,South Victoria,2019-06-09,07:00,2019-06-09,07:30,2019-06-09 07:02:00,NaT,ECLWQX,other problem,2,E95,37,45,A,"['Elizabeth Garcia_75_37_45_cp', 'Melanie Blac...",[],45.0,NaN,2019-06-09,6,30.0,NaN,2019-06-09 07:00:00,2019-06-09 07:30:00,0,0,0,0,0,0,True


In [46]:
# Since observing the last duplicate shows no real insight as to which column to drop, the even one randomly chosen will
# be dropped.
to_drop = investigation.index[0]
flight.drop(to_drop, inplace=True)

In [47]:
# Refind duplicates existing on registration plate number, flight number, date, time - checking there are truly none remaining
flight = flight_finds_duplicates(flight, ['ac_registration', 'fn_number', 'dep_sched_datetime', 'arr_sched_datetime'])
investigation = flight.loc[flight['duplicate_row'] == True]
investigation.shape

(0, 35)

## Train-test split on Flight data

In [48]:
# Since Flight has been deduped, an additional classification of deciles can be added on the total flight count.

def _adds_decile(df):
    investigation = df[['ac_registration', df.columns[0]]].groupby('ac_registration').count().reset_index(drop=False)
    investigation.columns = ['ac_registration', 'count']
    investigation = investigation.sort_values('count', ascending=False)

    # Define percentile boundaries
    decile = np.percentile(investigation['count'].values, [10, 20, 30, 40, 50, 60, 70, 80, 90])
    decile_labels = np.digitize(investigation['count'].values, decile, right=True)
    investigation['decile'] = decile_labels

    mydict = {key:value for (key, value) in zip(investigation['ac_registration'].to_list(), investigation['decile'].to_list())}

    # Map to df file
    df['decile'] = df['ac_registration'].map(mydict)
    return df

In [49]:
flight = _adds_decile(flight)

In [50]:
# Reseleecting columns
flight = flight[['day_of_origin', 'leg_no', 'fn_number', 'ac_registration', 'dep_ap_sched', 'arr_ap_sched', 'dep_sched_datetime',
                 'arr_sched_datetime', 'm_offblockdt', 'm_onblockdt', 'day_of_week','change_reason_code', 'dep_delay', 'Ac_Type_Code', 'trans_time',
                 'sched_trans_time', 'Crew_Group', 'TLC_trans', 'crew_type_change', 'Sched_Groundtime', 'Act_Groundtime', 'fn_carrier',
                 'sched_duration', 'actual_duration', 'last_flight_indicator', 'first_flight_indicator', 'dep_sched_time', 'arr_sched_time',
                 'dep_sched_date', 'arr_sched_date', 'last_and_blank_act_gt', 'first_and_blank_act_gt', 'last_and_blank_sched_gt',
                 'first_and_blank_sched_gt', 'decile']]

### Rationale for Train-test split

In [51]:
# Want to check that there is sufficient representation across the data to perform the train-test split with decile and day of the week combined.
# Looking for a situation where there is a count of at least 3 entries per day-decile combination so that a representative train-test split can be performed.
df_copy = flight.copy(deep=True)
df_copy = df_copy.reset_index(drop=False)
_sample = df_copy[['decile', 'day_of_origin', 'day_of_week', df_copy.columns[0]]].groupby(by=['decile', 'day_of_week', 'day_of_origin']).count().reset_index(drop=False)
_sample_group = _sample.groupby(by=['decile', 'day_of_week']).count().reset_index(drop=False)
_sample_group = _sample_group[['decile', 'day_of_week', 'index']]
_sample_group.columns = ['decile', 'day_of_week', 'count']
_sample_group = _sample_group.sort_values(by='count', ascending=True)
_sample_group['decile_and_day_of_week'] = _sample_group['decile'].astype('str') + '_' + _sample_group['day_of_week'].astype('str')

fig = px.bar(_sample_group, x="decile_and_day_of_week", y="count", color="count", title="Number of rows per total flight count Decile grouping and day of the week")
fig.update(layout_coloraxis_showscale=False)
fig.update_layout(title=None)
fig.write_image(images_folder + "/data_representation.png")
fig.show()

In [52]:
# As above indicates that there are at least 4 aircraft per decile and day of week combination, can proceed.

def _creates_train_test_split(df):
    # Initialize empty lists for indices for train and test sets
    mylist_i, mylist_ii = [], []
    df_copy = df.copy(deep=True)
    df_copy = df_copy.reset_index(drop=False)

    # Group by 'ac_registration', 'day_of_origin', 'day_of_week' and count these to find basis of a representative split
    _sample = df_copy[['ac_registration', 'day_of_origin', 'day_of_week', df_copy.columns[0]]].groupby(by=['ac_registration',
                                                                                                           'day_of_week', 'day_of_origin']).count().reset_index(drop=False)

    # 40% for the test portion
    _sample_group = _sample.groupby(by=['ac_registration', 'day_of_week'], group_keys=False).apply(lambda x: x.sample(frac=0.4, random_state=1))
    _sample_group = _sample_group.sort_index()

    all_details = pd.merge(_sample, _sample_group, how='outer', indicator=True)

    # Separate train and test sets
    train_summary = all_details[all_details['_merge'] == 'left_only'].drop('_merge', axis=1)
    test_summary = all_details[all_details['_merge'] == 'both'].drop('_merge', axis=1)

    # ac_registration and day_of_origin removed so can get the matching row indices (train dataset)
    ac_registration = train_summary['ac_registration'].to_list()
    day_of_origin = train_summary['day_of_origin'].to_list()

    # Get indices of matching rows
    for ac_reg, day in zip(ac_registration, day_of_origin):
        subset = df_copy.loc[(df_copy['ac_registration'] == ac_reg) & (df_copy['day_of_origin'] == day)].index.tolist()
        mylist_i.append(subset)

    # repeated for test dataset
    ac_registration = test_summary['ac_registration'].to_list()
    day_of_origin = test_summary['day_of_origin'].to_list()

    # Get indices of matching rows
    for ac_reg, day in zip(ac_registration, day_of_origin):
        subset = df_copy.loc[(df_copy['ac_registration'] == ac_reg) & (df_copy['day_of_origin'] == day)].index.tolist()
        mylist_ii.append(subset)

    mylist_i = [v for val in mylist_i for v in val]
    mylist_ii = [v for val in mylist_ii for v in val]
    mylist_i.sort()
    mylist_ii.sort()

    # Select rows for train and test sets from matching indices
    train = df_copy.iloc[mylist_i]
    test = df_copy.iloc[mylist_ii]

    # Ensure that each ac_registration in train set has a different day_of_origin compared to test set. Want to select entire days so can get the full picture of the flights for an ac_reg for that day.
    for ac_reg in set(train['ac_registration']):
        day_subset_train = set(train[(train['ac_registration'] == ac_reg)]['day_of_origin'])
        day_subset_test = set(test[(test['ac_registration'] == ac_reg)]['day_of_origin'])
        assert not day_subset_train.issubset(day_subset_test), "Error: Elements of set are present in other set"

    # Drop the index from both
    train.drop(columns=['index'], inplace=True)
    test.drop(columns=['index'], inplace=True)

    return train, test

In [53]:
flight_train, flight_test = _creates_train_test_split(flight)

In [54]:
pkl_path = '/content/drive/MyDrive/01_Data/03_Modelling/'

In [55]:
# Save deduped test flight file to pickle format for model.
with open(pkl_path + 'flight_test.pkl', 'wb') as file:
    pickle.dump(flight_test, file)

# Save deduped train flight file to pickle format for model.
with open(pkl_path + 'flight_train.pkl', 'wb') as file:
    pickle.dump(flight_train, file)

In [56]:
# The train flight file will be used to select the same days from the ground train file and split out the test file as well.

## Data Cleansing: Flight Train Data

### Dropping redundant Columns

In [57]:
# fn_carrier analysed for the categories contained within
to_plot = flight_train['fn_carrier'].value_counts(normalize=True)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=to_plot.index,
    x=to_plot.values,
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.show()

In [58]:
# Drop 'fn_carrier' as the information is redundant. Add 'day_of_origin' column to assist with analysis
flight_train = flight_train.drop(['fn_carrier'], axis=1)

# Drop 'dep_sched_date', 'dep_sched_time', 'arr_sched_date', 'arr_sched_time' because data in 'dep_sched_datetime' and 'arr_sched_datetime'
# respectively
flight_train.drop(['dep_sched_date', 'dep_sched_time', 'arr_sched_date', 'arr_sched_time'], axis=1, inplace=True)

# Sort by ac_reg, then by dep_sched_datetime for window calculations per ac_reg over a timeframe
flight_train = flight_train.sort_values(['ac_registration', 'dep_sched_datetime'], ascending = [True, True]).reset_index(drop=True)

### Imputing missing values for m_onblockdt, actual_duration

**Imputing the missing values for m_onblockdt: prior investigation**         

In [59]:
# Evaluate the time difference in minutes between 'arr_sched_time' and 'm_onblockdt' (expected arriving time versus actual arriving time)
# This is effectively the 'arrival delay' (we already have the 'departure delay')
flight_train['arr_delay_imputed'] = flight_train['m_onblockdt'] - flight_train['arr_sched_datetime']
flight_train['arr_delay_imputed'] = flight_train['arr_delay_imputed'] / np.timedelta64(1, 'm')

# Analyse the distribution of this for non-null values
_for_boxplot = flight_train.loc[flight['m_onblockdt'].notnull(), ['arr_delay_imputed']]
boxplot_hist_plotter(_for_boxplot, 'arr_delay_imputed')

In [60]:
col_statistics(_for_boxplot, 'arr_delay_imputed')

arr_delay_imputed is right skewed (positively skewed)


,arr_delay_imputed
count,6373.000000
mean,21.031225
std,26.294647
min,-83.000000
25%,3.000000
50%,14.000000
75%,33.000000
max,197.000000
mode,8.000000
median,14.000000


In [61]:
# From the above it can be seen that there is a bimodal distribution, from further investigation it appears that the reason
# for this is that 'change_reason_code' has separate distinct modes (as shown below).

# Analyse the distribution of this for non-null values
_for_boxplot = flight_train.loc[flight_train['m_onblockdt'].notnull(), ['arr_delay_imputed', 'change_reason_code']]

fig = make_subplots(rows=2, cols=2, subplot_titles=("Other problem", "Technical problem", "Rotational problem", "No reason"))

o_prob = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'other problem'][['arr_delay_imputed']]
fd_o_prob = freedman_diaconis(o_prob, 'arr_delay_imputed')

tech_prob = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'technical problem'][['arr_delay_imputed']]
fd_tech_prob = freedman_diaconis(tech_prob, 'arr_delay_imputed')

rot_prob = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'rotational problem'][['arr_delay_imputed']]
fd_rot_prob = freedman_diaconis(rot_prob, 'arr_delay_imputed')

no_reason = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'no reason'][['arr_delay_imputed']]
fd_no_reason = freedman_diaconis(no_reason, 'arr_delay_imputed')

fig.add_trace(
        go.Histogram(x=o_prob['arr_delay_imputed'], nbinsy=fd_o_prob, marker=dict(color=px.colors.qualitative.Bold[0])),
        row=1, col=1
    )
fig.add_trace(
        go.Histogram(x=tech_prob['arr_delay_imputed'], nbinsy=fd_tech_prob, marker=dict(color=px.colors.qualitative.Bold[2])),
        row=1, col=2
    )
fig.add_trace(
        go.Histogram(x=rot_prob['arr_delay_imputed'], nbinsy=fd_rot_prob, marker=dict(color=px.colors.qualitative.Bold[3])),
        row=2, col=1
    )
fig.add_trace(
        go.Histogram(x=no_reason['arr_delay_imputed'], nbinsy=fd_no_reason, marker=dict(color=px.colors.qualitative.Bold[1])),
        row=2, col=2
    )

fig.update_layout(height=600, width=1000, title_text='Histograms of arr_delay_imputed by Change reason code', showlegend=False)
fig.update_layout(title=None)
fig.write_image(images_folder + "/arr_delay_imputed.png")
fig.show()

In [62]:
# Analyse this in terms of the statistics per reason
mylist = []
for name, var in zip(['Other problem', 'Technical problem', 'Rotational problem', 'No reason'], [o_prob, tech_prob, rot_prob, no_reason]):
    new_df = col_statistics(var, 'arr_delay_imputed', print_skew=False)
    new_df.columns = [name]
    mylist.append(new_df)
summary = pd.concat(mylist, axis=1)
summary

,Other problem,Technical problem,Rotational problem,No reason
count,3981.000000,1082.000000,870.000000,478.000000
mean,9.416981,62.366913,37.928736,-6.106695
std,12.821129,23.246126,14.583263,7.631387
min,-83.000000,-15.000000,1.000000,-31.000000
25%,1.000000,54.000000,27.000000,-11.000000
50%,9.000000,64.000000,37.000000,-6.000000
75%,16.000000,77.000000,47.000000,-2.000000
max,197.000000,149.000000,106.000000,23.000000
mode,9.000000,61.000000,32.000000,-5.000000
median,9.000000,64.000000,37.000000,-6.000000


In [63]:
# The above shows that 'change_reason_code = Other problem' is skewing the overall data due to its high record count and separate mean.
# In fact, each change reason has its own distinct mean, mode and median.
# This will be revisited as it indicates a type of severity rank in terms of delay to the change_reason_codes.

In [64]:
# it would make sense to use the existing flight_train_df duration for each change_reason_code, to impute the
# missing values for the missing m_onblockdt.
investigation = flight_train.loc[flight['m_onblockdt'].isna()][['Ac_Type_Code' ,'change_reason_code', 'day_of_week']]
investigation.value_counts()

Ac_Type_Code  change_reason_code  day_of_week
320           other problem       4              6
              technical problem   6              4
DH4           other problem       0              4
                                  3              2
320           other problem       6              2
              technical problem   2              2
              other problem       2              2
DH4           other problem       1              2
320           other problem       0              1
E95           other problem       0              1
              no reason           0              1
DH4           technical problem   1              1
              rotational problem  5              1
                                  3              1
                                  0              1
320           technical problem   4              1
                                  0              1
              rotational problem  4              1
                                  3 

In [65]:
# Add column indicating if 'arr_delay_imputed' is null.
flight_train['arr_delay_imputed_null'] = flight_train['m_onblockdt'].isna().astype('int')

# Create a validation dataset using a stratified method
stratified_summary = flight_train[flight_train['arr_delay_imputed_null'] == 0].groupby(['change_reason_code', 'Ac_Type_Code', 'day_of_week'],
                                                                                             group_keys=False)[flight_train.columns[0]].count().reset_index()
stratified_summary.columns = ['change_reason_code', 'Ac_Type_Code', 'day_of_week','count']
stratified_summary['rows_to_random_sample'] = (stratified_summary['count'] * 0.2).astype('int')

mytrainlist, myvalidlist = [], []
for idx, row in stratified_summary.iterrows():
    subset = flight_train.loc[(flight_train['change_reason_code'] == row.change_reason_code) & (
        flight_train['Ac_Type_Code'] == row.Ac_Type_Code) & (flight_train['day_of_week'] == row.day_of_week) & (
        flight_train['arr_delay_imputed_null'] == 0)] # Only want non-nulls for validation set

    random = np.random.choice(subset.shape[0] - 1, row.rows_to_random_sample, replace=False)
    idx = subset.index[random].sort_values(ascending=True)
    subset = subset.reset_index(drop=False)
    _flight_train_validation = subset.loc[subset['index'].isin(idx)]
    _flight_train = subset.loc[~subset['index'].isin(idx)]

    # Adding the rest of the data (null) to the train df
    subset2 = flight_train.loc[(flight_train['change_reason_code'] == row.change_reason_code) & (
        flight_train['Ac_Type_Code'] == row.Ac_Type_Code) & (flight_train['day_of_week'] == row.day_of_week) & (
        flight_train['arr_delay_imputed_null'] == 1)]

    mytrainlist.append(_flight_train)
    mytrainlist.append(subset2)
    myvalidlist.append(_flight_train_validation)

_flight_train = pd.concat(mytrainlist, axis=0)
_flight_train_valid_df = pd.concat(myvalidlist, axis=0)

_flight_train.drop(columns=['index'], inplace=True)
_flight_train_valid_df.drop(columns=['index'], inplace=True)

_flight_train = _flight_train.reset_index(drop=False)

val_perc = _flight_train_valid_df.shape[0] / (_flight_train.shape[0] + _flight_train_valid_df.shape[0])
print(f'Validation set percentage: {val_perc:.2%}')

# Need to separate the training data into 2 (null and non-null), since the model can't train on null data.
# Use the null arr_delay_imputed as the prediction set.
mydf1 = _flight_train.loc[_flight_train['arr_delay_imputed_null'] == 0][['index', 'change_reason_code', 'trans_time', 'Ac_Type_Code', 'day_of_week', 'arr_delay_imputed']]
mydf2 = _flight_train.loc[_flight_train['arr_delay_imputed_null'] == 1][['index', 'change_reason_code', 'trans_time', 'Ac_Type_Code', 'day_of_week']]

Validation set percentage: 19.29%


In [66]:
# Will impute the arr_delay_imputed where missing with a preliminary model using stochastic gradient descent.
# This is because it provides regularization options within so we have a guard against overfitting.

# Training data
X = mydf1[['change_reason_code', 'Ac_Type_Code']]
y = mydf1['arr_delay_imputed']

categorical_cols = ['change_reason_code', 'Ac_Type_Code']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Set a random seed so the model will reproduce in the class object - see file ClassFns
random_seed = 42
np.random.seed(random_seed)

# Create preprocessor for one hot encoding of the categorical columns and the standard scaling steps
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        # ('num', numerical_transformer, numerical_cols)
    ])

# Pipeline with SGDRegressor and the preprocessing
reg = make_pipeline(preprocessor, SGDRegressor(max_iter=1000, penalty='l2', alpha=0.0001, shuffle=True, tol=1e-3,
                                               learning_rate='invscaling', random_state=random_seed))

# Fit the model
reg.fit(X, y)

# Save for use with test data
joblib.dump(reg, pkl_path + 'sgd_pipeline.pkl')

# Testing data
# X_pred = mydf2[['change_reason_code', 'Ac_Type_Code', 'day_of_week', 'trans_time']]
X_pred = mydf2[['change_reason_code', 'Ac_Type_Code']]
y_pred = reg.predict(X_pred)

mydf2['y_pred'] = y_pred

# Validation data
# X_val = _flight_train_valid_df[['change_reason_code', 'Ac_Type_Code', 'day_of_week', 'trans_time']]
X_val = _flight_train_valid_df[['change_reason_code', 'Ac_Type_Code']]
y_val = _flight_train_valid_df['arr_delay_imputed']

y_val_pred = reg.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f'Root Mean Squared Error on Validation Set: {rmse}')

range_of_target_variable = _flight_train_valid_df['arr_delay_imputed'].max() - _flight_train_valid_df['arr_delay_imputed'].min()
percentage_rmse = (rmse / range_of_target_variable) * 100
print(f'Percentage RMSE relative to the range: {percentage_rmse:.2f}%')

Root Mean Squared Error on Validation Set: 15.143098255803014
Percentage RMSE relative to the range: 6.85%


In [67]:
# Replace the 'arr_delay_imputed' with y_pred where necessary.
mydict = dict()
for index, row in flight_train.iterrows():
    if index in mydf2['index'].to_list():
       mydict.update({index : mydf2.loc[mydf2['index'] == index]['y_pred'].values[0]})
    else:
        mydict.update({index : row.arr_delay_imputed})

flight_train['arr_delay_imputed'] = flight_train.index.map(mydict)
flight_train['arr_delay_imputed'] = pd.to_timedelta(flight_train['arr_delay_imputed'], unit='m')

# Reverse calculate the m_onblockdt
flight_train['m_onblockdt_imputed'] = flight_train['arr_delay_imputed'] + flight_train['arr_sched_datetime']

In [68]:
# Re-calculate the 'arr_delay_imputed' to use the 'm_onblockdt_imputed' column instead of the 'm_onblockdt'.
def impute_arr_delay(df):
    df['arr_delay_imputed'] = df['m_onblockdt_imputed'] - df['arr_sched_datetime']
    df['arr_delay_imputed'] = df['arr_delay_imputed'] / np.timedelta64(1, 'm')
    df['arr_delay_imputed'] = df['arr_delay_imputed'].astype('int')
    return df

flight_train = impute_arr_delay(flight_train)

In [69]:
# Graphically view how the imputed data appears per change reason code.
# Imputed values tend towards the mean for each change reason code - this seems reasonable.
# The tendency towards the mean also indicates consistent behavior across the different change reason codes.

investigation = flight_train[['arr_delay_imputed', 'm_onblockdt','change_reason_code']]
investigation['imputed'] = np.where(investigation['m_onblockdt'].isnull(), 1, 0)

fig = px.strip(investigation, x='change_reason_code', y='arr_delay_imputed', color='imputed', stripmode='overlay',
               color_discrete_sequence=px.colors.qualitative.Safe, title="Imputed & unimputed arrival_delay values to show if in same distribution")
fig.update_traces(jitter=1.0, marker={'size': 3.5})
fig.update_layout(title=None)
fig.write_image(images_folder + "/arr_delay_imputed_v_unimputed.png")
fig.show()

In [70]:
# Can also check it behaves as expected since arr_sched_datetime and m_onblockdt_imputed are expected to be correlated.
# We can use arr_sched_datetime as a comparison since arr_delay_imputed is calculated from df['m_onblockdt_imputed'] - df['arr_sched_datetime']

fig = px.scatter(x=flight_train[flight_train['arr_delay_imputed_null'] == 1]['arr_sched_datetime'],
                 y=flight_train[flight_train['arr_delay_imputed_null'] == 1]['m_onblockdt_imputed'],
                color_discrete_sequence=px.colors.qualitative.Bold, title="Correlation for imputed m_onblockdt and arr_sched_datetime")
fig.show()

In [71]:
# Thus it is shown that this is an effective method and keeps within the range of unimputed values.

### Further investigation of change_reason_code and delays

In [72]:
# The work above indicates that the dep_delay column has a relationship with the change_reason_code. This is more fully investigated
# here.

# Analysing the distribution of the non-null dep_delay values, per change_reason_code.

_for_boxplot = flight_train.loc[(flight_train['dep_delay'].notnull()) & (flight_train['change_reason_code'].notnull()), ['dep_delay', 'change_reason_code']]

o_prob = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'other problem'][['dep_delay']]
fd_o_prob = freedman_diaconis(o_prob, 'dep_delay')

tech_prob = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'technical problem'][['dep_delay']]
fd_tech_prob = freedman_diaconis(tech_prob, 'dep_delay')

rot_prob = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'rotational problem'][['dep_delay']]
fd_rot_prob = freedman_diaconis(rot_prob, 'dep_delay')

no_reason = _for_boxplot.loc[_for_boxplot['change_reason_code'] == 'no reason'][['dep_delay']]

In [73]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(x=o_prob['dep_delay'], name='Other problem', nbinsy=fd_o_prob, marker=dict(color=px.colors.qualitative.Bold[0]))
                )
fig.add_trace(
    go.Histogram(x=tech_prob['dep_delay'], name='Technical problem', nbinsy=fd_tech_prob, marker=dict(color=px.colors.qualitative.Bold[1]))
                )
fig.add_trace(
    go.Histogram(x=rot_prob['dep_delay'], name='Rotational problem', nbinsy=fd_rot_prob, marker=dict(color=px.colors.qualitative.Bold[2]))
                )
fig.add_trace(
    go.Histogram(x=no_reason['dep_delay'], name='No reason', marker=dict(color=px.colors.qualitative.Bold[3]))
                )

# Overlay histograms
fig.update_layout(barmode='overlay')

# Reduce opacity for overlay
fig.update_traces(opacity=0.75)
fig.update_layout(height=500, width=1000, title_text='Histograms of departure delay time by Change reason code')
fig.show()

In [74]:
# Separate histograms per change reason code

fig = make_subplots(rows=2, cols=2, subplot_titles=("Other problem", "Technical problem", "Rotational problem", "No reason"))

fig.add_trace(
        go.Histogram(x=o_prob['dep_delay'], nbinsy=fd_o_prob, marker=dict(color=px.colors.qualitative.Bold[0])),
        row=1, col=1
    )
fig.add_trace(
        go.Histogram(x=tech_prob['dep_delay'], nbinsy=fd_tech_prob, marker=dict(color=px.colors.qualitative.Bold[1])),
        row=1, col=2
    )
fig.add_trace(
        go.Histogram(x=rot_prob['dep_delay'], nbinsy=fd_rot_prob, marker=dict(color=px.colors.qualitative.Bold[2])),
        row=2, col=1
    )
fig.add_trace(
        go.Histogram(x=no_reason['dep_delay'], marker=dict(color=px.colors.qualitative.Bold[3])),
        row=2, col=2
    )

fig.update_layout(height=600, width=1000, title_text='Histograms of dep_delay by Change reason code', showlegend=False)
fig.show()

In [75]:
# As from the above it is noted that flights departing early or on-time (0 delay) are assigned 'no reason'.
# Those that are 'Other problem' have the highest record count but are also of the shortest duration, delay-wise.
# Those that are 'Rotational problem' cause a delay of approximately 40 minutes.
# Those that are a 'Technical problem' cause a longer delay of approximately 60 minutes.

# Thus we can apply some sort of order or ranking to these change reason codes in terms of severity for the time delay arising, where
# 0 : No reason; 1 : Other problem, 2 : Rotational problem, 3 : Technical problem.

### TLC_trans column splitting, cleaning and investigation

In [76]:
def split_TLC_trans(df):
    # Split data contained into a list
    mylist = []
    for item in df['TLC_trans']:
        mylist_ = []
        split_item = item.split(",")
        TLC_trans_1, TLC_trans_2, TLC_trans_3, TLC_trans_4 = [], [], [], []
        for index, entry in enumerate(split_item):
            sub_entry = entry.split("_")
            TLC_trans_1.append(sub_entry[1])
            TLC_trans_2.append(sub_entry[2])
            TLC_trans_3.append(sub_entry[3])
            TLC_trans_4.append(sub_entry[4])
        TLC_trans_1_set = set(TLC_trans_1)
        TLC_trans_2_set = set(TLC_trans_2)
        TLC_trans_3_set = set(TLC_trans_3)

        count_1 = [TLC_trans_1.count(item) for item in TLC_trans_1_set]
        count_2 = [TLC_trans_2.count(item) for item in TLC_trans_2_set]
        count_3 = [TLC_trans_3.count(item) for item in TLC_trans_3_set]

        split_1 = {i:ii for i, ii in zip(TLC_trans_1_set, count_1)}
        split_1 = [max(split_1, key=split_1.get)]
        split_2 = {i:ii for i, ii in zip(TLC_trans_2_set, count_2)}
        split_2 = [max(split_2, key=split_2.get)]
        split_3 = {i:ii for i, ii in zip(TLC_trans_3_set, count_3)}
        split_3 = [max(split_3, key=split_3.get)]
        split_1.extend(split_2)
        split_1.extend(split_3)
        split_1.extend([TLC_trans_4])
        mylist.append(split_1)

    # Adds the data in the list to df
    df['TLC_trans_1'] = [item[0] for item in mylist]
    df['TLC_trans_2'] = [item[1] for item in mylist]
    df['TLC_trans_3'] = [item[2] for item in mylist]
    df['TLC_trans_4'] = [item[3] for item in mylist]

    # Replaces NaN with zero
    df['TLC_trans_1'] = df['TLC_trans_1'].replace('nan', 0).astype('int')
    df['TLC_trans_2'] = df['TLC_trans_2'].replace('nan', 0).astype('int')
    df['TLC_trans_3'] = df['TLC_trans_3'].replace('nan', 0).astype('int')
    return df

In [77]:
flight_train = split_TLC_trans(flight_train)
flight_train.head()

,day_of_origin,leg_no,fn_number,ac_registration,dep_ap_sched,arr_ap_sched,dep_sched_datetime,arr_sched_datetime,m_offblockdt,m_onblockdt,day_of_week,change_reason_code,dep_delay,Ac_Type_Code,trans_time,sched_trans_time,Crew_Group,TLC_trans,crew_type_change,Sched_Groundtime,Act_Groundtime,sched_duration,actual_duration,last_flight_indicator,first_flight_indicator,last_and_blank_act_gt,first_and_blank_act_gt,last_and_blank_sched_gt,first_and_blank_sched_gt,decile,arr_delay_imputed,arr_delay_imputed_null,m_onblockdt_imputed,TLC_trans_1,TLC_trans_2,TLC_trans_3,TLC_trans_4
0,2019-06-01,272024970,EC3292,ECLBAX,New Jessica,East Carmen,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,5,other problem,25,320,0,0,Start,"['Sean Weeks_nan_nan_nan_ca', 'Tony Lloyd_nan_...",[],95.0,94.0,200.0,191.0,0,1,0,0,0,0,1,16,0,2019-06-01 07:01:00,0,0,0,"[ca', ca', ca', cp', cp', ca']]"
1,2019-06-01,272022230,EC3257,ECLBAX,East Carmen,South Nathaniel,2019-06-01 08:20:00,2019-06-01 10:35:00,2019-06-01 08:35:00,2019-06-01 10:41:00,5,other problem,15,320,60,60,B,"['Toni Rodriguez_48_60_60_ca', 'Rachel Smith_4...",[],75.0,120.0,135.0,126.0,0,0,0,0,0,0,1,6,0,2019-06-01 10:41:00,48,60,60,"[ca', cp', ca', ca', cp', ca']]"
2,2019-06-01,272212848,EC3258,ECLBAX,South Nathaniel,East Carmen,2019-06-01 11:50:00,2019-06-01 14:15:00,2019-06-01 12:41:00,2019-06-01 14:52:00,5,rotational problem,51,320,68,75,A,"['Toni Rodriguez_127_68_75_ca', 'Rachel Smith_...",[],80.0,86.0,145.0,131.0,0,0,0,0,0,0,1,37,0,2019-06-01 14:52:00,127,68,75,"[ca', cp', ca', ca', cp', ca']]"
3,2019-06-01,271997824,EC3021,ECLBAX,East Carmen,Joneshaven,2019-06-01 15:35:00,2019-06-01 17:00:00,2019-06-01 16:18:00,2019-06-01 17:32:00,5,rotational problem,43,320,64,70,B2,"['Tyler Morris_224_64_70_ca', 'Sergio Cummings...","['ca', 'cp', 'ca', 'ca', 'ca']",50.0,32.0,85.0,74.0,0,0,0,0,0,0,1,32,0,2019-06-01 17:32:00,224,64,70,"[ca', cp', cp', ca', ca', ca']]"
4,2019-06-01,271998033,EC3022,ECLBAX,Joneshaven,East Carmen,2019-06-01 17:50:00,2019-06-01 19:10:00,2019-06-01 18:04:00,2019-06-01 19:14:00,5,other problem,14,320,42,50,A,"['Sergio Cummings_279_42_50_cp', 'Jackie Black...",[],NaN,NaN,80.0,70.0,1,0,1,0,1,0,1,4,0,2019-06-01 19:14:00,279,42,50,"[cp', ca', ca', ca', ca', cp']]"


In [78]:
# Observe that 'TLC_trans_2' and 'TLC_trans_3' appear equal to 'trans_time' and 'sched_trans_time' respectively, double-check that this
# is indeed the case
mylist = []
for idx, row in flight_train.iterrows():
    running_sum = 0
    equality_check_trans_time = row.TLC_trans_2 == row.trans_time
    equality_check_sched_trans_time = row.TLC_trans_3 == row.sched_trans_time
    if equality_check_trans_time is True:
        running_sum += 1
    if equality_check_sched_trans_time is True:
        running_sum += 1
    mylist.append((idx, running_sum))

flight_train['TLC_trans_2_3_V_trans_sched_trans_time'] = [str(item[1]) for item in mylist]

In [79]:
# Double-check the results

summary_1 = flight_train['TLC_trans_2_3_V_trans_sched_trans_time'].value_counts(normalize=True)
summary_1.columns = ['proportion']
summary_2 = flight_train['TLC_trans_2_3_V_trans_sched_trans_time'].value_counts()
summary_2.columns = ['Number_matching']
summary = pd.concat([summary_1, summary_2], axis=1)
summary = summary.reset_index()
summary.columns = ['index', 'proportion', 'Number_matching']
summary['index'] = summary['index'].astype('str')
summary['Perc'] = [str(math.ceil(item[1] * 100)) + '%' for item in summary.values]
summary = summary.sort_values(by='index', ascending=False)

fig = px.bar(summary, x="proportion", y="index", color='Number_matching', orientation='h',
             text='Perc', height=400, color_discrete_sequence=px.colors.qualitative.Bold,
             title='Matches of TLC_trans_2, TLC_trans_3 to trans_time and sched_trans_time respectively')

fig.update_layout(showlegend=False)
fig.show()

In [80]:
# Since a match exists in approximately 94%, checking what the issue is for the other 6%.
investigation = flight_train.loc[flight_train['TLC_trans_2_3_V_trans_sched_trans_time']!= '2']

summary = investigation['TLC_trans_2'].value_counts(normalize=True).sort_values().sort_index().reset_index()
summary['Perc'] = [str(math.ceil(item[1] * 100)) + '%' for item in summary.values]

fig = px.bar(summary, x=summary.index, y="TLC_trans_2", color='TLC_trans_2', orientation='v', color_continuous_scale=px.colors.sequential.Magenta,
             text='Perc', height=400, title='')
fig.update_layout(showlegend=False)
fig.show()

In [81]:
summary = investigation['TLC_trans_3'].value_counts(normalize=True).sort_values().sort_index().reset_index()
summary['Perc'] = [str(math.ceil(item[1] * 100)) + '%' for item in summary.values]

fig = px.bar(summary, x=summary.index, y="TLC_trans_3", color='TLC_trans_3', orientation='v', color_continuous_scale=px.colors.sequential.Tealgrn,
             text='Perc', height=400, title='')
fig.update_layout(showlegend=False)
fig.show()

In [82]:
# This shows that while the flight file might have the trans_time and sched_trans_time fields populated, the 'nan' field in
# the split is obviously not matching - thus if no match exists between trans_time, sched_trans_time and the split we will
# rather choose the trans_time, sched_trans_time field if it exists.

In [83]:
# Due to the inconsistent results when splitting TLC_trans, these columns will be dropped before the model is created.

In [84]:
# Reselecting the required columns
flight_train = flight_train[['day_of_origin', 'leg_no', 'fn_number', 'ac_registration', 'dep_ap_sched', 'arr_ap_sched', 'dep_sched_datetime',
                 'arr_sched_datetime', 'm_offblockdt', 'm_onblockdt', 'day_of_week', 'change_reason_code', 'dep_delay', 'arr_delay_imputed',
                 'arr_delay_imputed_null', 'Ac_Type_Code', 'trans_time', 'sched_trans_time', 'Crew_Group', 'TLC_trans', 'crew_type_change',
                 'Sched_Groundtime', 'Act_Groundtime', 'last_flight_indicator', 'first_flight_indicator', 'm_onblockdt_imputed', 'TLC_trans_1',
                 'TLC_trans_2', 'TLC_trans_3', 'TLC_trans_4', 'sched_duration', 'actual_duration', 'last_and_blank_act_gt', 'first_and_blank_act_gt',
                 'last_and_blank_sched_gt', 'first_and_blank_sched_gt', 'decile']]
flight_train.sort_values(['ac_registration', 'dep_sched_datetime'], ascending = [True, True]).reset_index(drop=True)
flight_train.head(1)

,day_of_origin,leg_no,fn_number,ac_registration,dep_ap_sched,arr_ap_sched,dep_sched_datetime,arr_sched_datetime,m_offblockdt,m_onblockdt,day_of_week,change_reason_code,dep_delay,arr_delay_imputed,arr_delay_imputed_null,Ac_Type_Code,trans_time,sched_trans_time,Crew_Group,TLC_trans,crew_type_change,Sched_Groundtime,Act_Groundtime,last_flight_indicator,first_flight_indicator,m_onblockdt_imputed,TLC_trans_1,TLC_trans_2,TLC_trans_3,TLC_trans_4,sched_duration,actual_duration,last_and_blank_act_gt,first_and_blank_act_gt,last_and_blank_sched_gt,first_and_blank_sched_gt,decile
0,2019-06-01,272024970,EC3292,ECLBAX,New Jessica,East Carmen,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,5,other problem,25,16,0,320,0,0,Start,"['Sean Weeks_nan_nan_nan_ca', 'Tony Lloyd_nan_...",[],95.0,94.0,0,1,2019-06-01 07:01:00,0,0,0,"[ca', ca', ca', cp', cp', ca']]",200.0,191.0,0,0,0,0,1


### Imputing the 'crew_type_change' column

In [85]:
# Find the pattern of where the 'crew_type_change' column is used - appears to be when entire staff on airplane is changed
# but still need to confirm this.

investigation = flight_train.loc[flight_train['crew_type_change'] != '[]'][['crew_type_change']]
investigation_lagged = flight_train.iloc[investigation.index - 1][['crew_type_change']]

In [86]:
# Need the lines where 'crew_type_change' has a value, as well as the preceding line.
summary = investigation_lagged['crew_type_change'].value_counts(normalize=True).reset_index()
summary['Perc'] = [str(math.ceil(item[1] * 100)) + '%' for item in summary.values]

fig = px.bar(summary.loc[summary['crew_type_change'] != '[]'], x="crew_type_change", y="proportion", orientation='v', color_continuous_scale=px.colors.sequential.Agsunset,
             text='Perc', height=400, title='')
fig.update_layout(showlegend=False)
fig.update_coloraxes(showscale=False)
fig.show()

In [87]:
# Shows that majority of the data where crew_type_change has a value is preceded by an empty cell - supports the idea that
# an existing crew/crew member is replaced and this will then mean that 'crew_type_change' is populated. If the crew stays on for the
# next flight the value [] is used.

In [88]:
mylist = []
to_replace = ["_nan", "Mrs. ", "Dr. ", "Mr. ", "Miss ", "MD", "PhD", "DDS", "DVM", "Jr.", "Ms.", "II", "_\d+", "_-\d+", "]", "[", " '", "'"]
TLC_trans_cleaned_split = flight_train['TLC_trans'].array

for split in TLC_trans_cleaned_split:
    split_string = str(split)
    for item in to_replace:
        try:
            split_string = re.sub(item, "", split_string)
        except:
            split_string = split_string.replace(item, "")
    newlist = split_string.split(',')
    newlist.sort()
    mylist.append(newlist)

flight_train['TLC_trans_pos'] = mylist

In [89]:
mylist = []
for reg in OrderedSet(flight_train['ac_registration'].values):
    subset = flight_train.loc[flight_train['ac_registration'] == reg][['TLC_trans_pos']]
    subset_ord = subset.iloc[:-1].reset_index(drop=True)
    subset_lag = subset.iloc[1:].reset_index(drop=True)
    maximum = max(subset_ord.index)
    for x, y, idx in zip(subset_ord.values, subset_lag.values, subset_ord.index):
        diff = set(x.tolist()[0]).difference(y.tolist()[0])
        if diff:
            z = re.findall("_[a-z][a-z]",str(diff))
            z = re.sub("_","", str(z))
            mylist.append((idx, z))
        else:
            mylist.append((idx, "[]"))
        if idx == maximum:
            mylist.append((idx + 1, "[]"))

mylist = [ii for i, ii in mylist]

mylist_i = []
for string_line in mylist:
    string_line = re.sub("\[","", string_line)
    string_line = re.sub("\]", "", string_line)
    string_line = re.sub("\'", "", string_line)
    string_line = re.sub("\,", "", string_line)
    string_line = string_line.strip()
    new_list = y = string_line.split()
    new_list.sort()
    mylist_i.append(new_list)

flight_train['crew_type_change_imputed'] = mylist_i

In [90]:
# Drop crew_type_change column because replaced by crew_type_change_imputed.
# Drop TLC_trans_4 because crew_type_change_imputed better.
flight_train.drop(['TLC_trans_4', 'crew_type_change'], axis=1, inplace=True)

# Drop TLC_trans because TLC_trans_pos a cleaner version
flight_train.drop(['TLC_trans'], axis=1, inplace=True)

### Imputing 'departure delay' column to verify accuracy of 'dep_delay' column in existence

In [91]:
flight_train['dep_delay_imputed'] = flight_train['m_offblockdt'] - flight_train['dep_sched_datetime']
flight_train['dep_delay_imputed'] = flight_train['dep_delay_imputed'] / np.timedelta64(1, 'm')
flight_train['dep_delay_imputed_check'] = flight_train['dep_delay_imputed'] - flight_train['dep_delay']

assert flight_train['dep_delay_imputed_check'].all() == 0, "error for investigation"

In [92]:
# As this is the case, the columns can be dropped.
flight_train.drop(['dep_delay_imputed_check', 'dep_delay_imputed'], axis=1, inplace=True)

### Working with lagged calculations

In [93]:
# Since the data is arranged per ac_registration, in dep_sched_datetime ascending, calculating anything for a time is
# specific to the aircraft registration, per day

In [94]:
def calculates_time_diff_to_line_below(df, lagged_col, normal_col, new_col_name):
    mydict = dict()
    for reg in OrderedSet(df['ac_registration'].values):
        subset = df.loc[df['ac_registration'] == reg][[lagged_col, normal_col, 'last_flight_indicator']].reset_index(drop=False)
        subset_ord = subset.iloc[:-1].reset_index(drop=True)[normal_col]
        subset_lag = subset.iloc[1:].reset_index(drop=True)[lagged_col]
        subset['diff'] = (subset_lag - subset_ord) / np.timedelta64(1, 'm')
        subset['diff'] = subset['diff'].fillna(0)
        subset.loc[subset['last_flight_indicator'] == 1, 'diff'] = 0
        mydict.update({index: value for (index, value) in zip(subset['index'].values, subset['diff'].values)})
    df[new_col_name] =  df.index.map(mydict)
    df[new_col_name] = df[new_col_name].astype('int')
    return df

In [95]:
Sched_Groundtime = calculates_time_diff_to_line_below(flight_train, 'dep_sched_datetime', 'arr_sched_datetime', 'Sched_Groundtime_imputed')

Act_Groundtime = calculates_time_diff_to_line_below(flight_train, 'm_offblockdt', 'm_onblockdt_imputed', 'Act_Groundtime_imputed')

# Recalculate actual_duration to use imputed values
flight_train['actual_duration'] = creates_duration_col(flight_train, 'm_offblockdt', 'm_onblockdt_imputed')
flight_train['actual_duration'] = np.round(flight_train['actual_duration'],0)

### Analyse the distribution of Sched_Groundtime_imputed, Act_Groundtime_imputed

In [96]:
# Analyse the distribution of Sched_Groundtime_imputed
_for_boxplot = flight_train[['Sched_Groundtime_imputed']]
boxplot_hist_plotter(_for_boxplot, 'Sched_Groundtime_imputed')

In [97]:
col_statistics(_for_boxplot, 'Sched_Groundtime_imputed')

,Sched_Groundtime_imputed
count,6463.000000
mean,53.257775
std,57.173078
min,-120.000000
25%,40.000000
50%,45.000000
75%,60.000000
max,945.000000
mode,45.000000
median,45.000000


In [98]:
# Analyse the distribution of Act_Groundtime_imputed
_for_boxplot = flight_train[['Act_Groundtime_imputed']]
boxplot_hist_plotter(_for_boxplot, 'Act_Groundtime_imputed')

In [99]:
col_statistics(_for_boxplot, 'Act_Groundtime_imputed')

Act_Groundtime_imputed is right skewed (positively skewed)


,Act_Groundtime_imputed
count,6463.000000
mean,56.358038
std,69.007362
min,-249.000000
25%,0.000000
50%,49.000000
75%,80.000000
max,947.000000
mode,0.000000
median,49.000000


### Investigating and correcting 'Crew_Group' column

In [100]:
# Split time from date
investigation = flight_train[['Crew_Group', 'dep_sched_datetime']]
investigation['Time'] = pd.to_datetime(investigation['dep_sched_datetime']).dt.time
investigation['Hour'] = pd.to_datetime(investigation['dep_sched_datetime']).dt.hour

# Want a summary per Crew_Group, per Hour
summary = pd.pivot_table(investigation, index=['Crew_Group', 'Hour'], aggfunc="count")
summary = summary.reset_index()
summary = summary[['Crew_Group', 'Hour', 'Time']]
summary.rename(columns={'Time': 'Count'}, inplace=True)

In [101]:
# Analyse graphically
mylist = []
for name in ['Start', 'A', 'C', 'B', 'B2', '']:
    if name != '':
        x = summary.loc[summary['Crew_Group'] == name]['Hour'].to_list()
        y = summary.loc[summary['Crew_Group'] == name]['Count'].to_list()
        mylist.append(x)
        mylist.append(y)

fig = make_subplots(rows=3, cols=2, subplot_titles=('Start', 'A', 'B', 'B2', 'C',''))
fig.add_trace(go.Bar(x=mylist[0], y=mylist[1],
                    marker=dict(color=mylist[1], colorscale="Viridis")),
              1, 1)
fig.add_trace(go.Bar(x=mylist[2], y=mylist[3],
                    marker=dict(color=mylist[3], colorscale="Viridis")),
              1, 2)
fig.add_trace(go.Bar(x=mylist[4], y=mylist[5],
                    marker=dict(color=mylist[5], colorscale="Viridis")),
              2, 1)
fig.add_trace(go.Bar(x=mylist[6], y=mylist[7],
                    marker=dict(color=mylist[7], colorscale="Viridis")),
              2, 2)
fig.add_trace(go.Bar(x=mylist[8], y=mylist[9],
                    marker=dict(color=mylist[9], colorscale="Viridis")),
              3, 1)

fig.update_layout(height=1000, width=1000, title_text='Histograms of Hour assigned per Crew Group value', showlegend=False)
fig.update_coloraxes(showscale=False)
fig.show()

In [102]:
# This indicates that the data within this field does not correspond to what is expected by hour.
# The definition supplied:

# Crew Group: assignment of what happens to the whole crew after a flight
# o Start : First flight of day
# o A : all crew members stay on the aircraft for the next flight
# o B, B2: all crew members switch aircraft for the next flight
# o C: at least one crew member switches aircraft for the next flight

# The column can be simplified to 'change' and 'no change', i.e. A and everything that is not A respectively

In [103]:
mylist = []

for reg in OrderedSet(flight_train['ac_registration'].values):
    subset = flight_train.loc[flight_train['ac_registration'] == reg][['Crew_Group', 'crew_type_change_imputed']]
    subset_ord = subset.iloc[:-1].reset_index(drop=True)['crew_type_change_imputed']
    mylist_ = ['start']
    for val, idx in zip(subset_ord, subset_ord.index):
        if len(val) == 0:
            crew = 'no change'
        else:
            crew = 'change'
        mylist_.extend([crew])
    mylist.append(mylist_)

mylist = [i for item in mylist for i in item]

flight_train['Crew_Group_imputed'] = mylist

### Imputing the change_reason_code for missing values

In [104]:
# Previously noted that if delay = 0, 'no reason' can be used as the change reason code.
flight_train['change_reason_code_imputed'] = flight_train['change_reason_code'].where(flight_train['dep_delay'] != 0, 'no reason')

### TLC_trans_1 investigation

In [105]:
# The definition of the TLC_trans_1 column is not immediately obvious as with TLC_trans_2, TLC_trans_3 and thus will be
# investigated further to see if this can be ascertained.
_for_boxplot = flight_train[['TLC_trans_1']]
boxplot_hist_plotter(_for_boxplot, 'TLC_trans_1')

In [106]:
col_statistics(_for_boxplot, 'TLC_trans_1')

TLC_trans_1 is right skewed (positively skewed)


,TLC_trans_1
count,6463.000000
mean,100.028160
std,114.117052
min,0.000000
25%,0.000000
50%,48.000000
75%,198.500000
max,403.000000
mode,0.000000
median,48.000000


In [107]:
# The mode of 0 and lack of obvious distribution means that a function for this column is difficult to interpret.
# It will be dropped going forward.

flight_train.drop(columns=['TLC_trans_1', 'TLC_trans_2', 'TLC_trans_3'], inplace=True)
assert 'TLC_trans columns' not in flight_train.columns, 'TLC_trans_1 still in cols'

In [108]:
# Save final flight file to pickle format for arr_leg_outbound lookups.
_flight_train = flight_train[['dep_sched_datetime', 'fn_number', 'dep_ap_sched', 'arr_ap_sched', 'ac_registration', 'leg_no']]
with open(pkl_path + '_flight_lookups.pkl', 'wb') as file:
    pickle.dump(_flight_train, file)

## Ground file: Deduplication and missing data

In [109]:
# Creating function to check duplicates
def ground_finds_duplicates(df, cols):
    _df = df.duplicated(subset=cols, keep=False)
    df['duplicate_row'] = _df
    df = df.sort_values(['ac_registration', 'sched_inbound_dep'], ascending = [True, True]).reset_index(drop=True)
    return df

In [110]:
# Find duplicates existing on registration plate number, flight number, date, time
ground = ground_finds_duplicates(ground, ['day_of_origin', 'ac_type', 'fn_number', 'ac_registration', 'dep_leg_inbound',
                                             'arr_leg_inbound', 'arr_leg_outbound', 'sched_inbound_dep', 'sched_inbound_arr'])
investigation = ground.loc[ground['duplicate_row'] == True]
investigation.shape

(56, 20)

In [111]:
investigation.head()

,day_of_origin,ac_type,fn_number,ac_registration,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,day_of_week,duplicate_row
637,2019-06-09,320,EC11860,ECLBNX,40,New Stacyburgh,East Carmen,South Haileyberg,2019-06-09 08:30:00,2019-06-09 10:30:00,2019-06-09 11:15:00,2019-06-09 15:35:00,45,272088101,272175531,15,NaN,17.0,6,True
638,2019-06-09,320,EC11860,ECLBNX,40,New Stacyburgh,East Carmen,South Haileyberg,2019-06-09 08:30:00,2019-06-09 10:30:00,2019-06-09 11:15:00,2019-06-09 15:35:00,45,272088101,272175531,17,NaN,17.0,6,True
639,2019-06-09,320,EC11860,ECLBNX,40,New Stacyburgh,East Carmen,South Haileyberg,2019-06-09 08:30:00,2019-06-09 10:30:00,2019-06-09 11:15:00,2019-06-09 15:35:00,45,272088101,272175531,15,NaN,17.0,6,True
640,2019-06-09,320,EC11860,ECLBNX,40,New Stacyburgh,East Carmen,South Haileyberg,2019-06-09 08:30:00,2019-06-09 10:30:00,2019-06-09 11:15:00,2019-06-09 15:35:00,45,272088101,272175531,17,NaN,17.0,6,True
650,2019-06-13,320,EC11860,ECLBNX,40,East Michael,East Carmen,Youngland,2019-06-13 10:30:00,2019-06-13 12:30:00,2019-06-13 13:20:00,2019-06-13 14:55:00,50,272088103,272020326,18,NaN,17.0,3,True


In [112]:
# shows the column/s with high cardinality
prints_differing_cols(investigation)

{'catering_duration': 52}

In [113]:
# Since it seems the only discrepancy is in the 'catering_duration' column - requires further investigation.

In [114]:
# Analyse the distribution of this for non-null values
_for_boxplot = ground.loc[ground['catering_duration'].notnull(), ['catering_duration']]
boxplot_hist_plotter(_for_boxplot, 'catering_duration')

In [115]:
col_statistics(_for_boxplot, 'catering_duration')

catering_duration is left skewed (negatively skewed)


,catering_duration
count,4173.000000
mean,21.959262
std,4.325588
min,15.000000
25%,18.000000
50%,22.000000
75%,26.000000
max,29.000000
mode,23.000000
median,22.000000


In [116]:
# Thus will replace with average duration across each duplicate row subset, then de-dupe.

In [117]:
def calcs_avg_catering_dur_dup_rows():
    mydict = dict()
    for entry in set(investigation['leg_inbound'].to_list()):
        avg = investigation.loc[investigation['leg_inbound'] == entry][['leg_inbound', 'catering_duration']].groupby('leg_inbound').mean().reset_index()
        mydict.update({avg['leg_inbound'].values[0] : avg['catering_duration'].values[0]})

    rest = {key: value for (key, value) in zip(ground['leg_inbound'].to_list(), ground['catering_duration'].to_list()) if key not in mydict.keys()}
    mydict.update(rest)
    ground['catering_duration'] = ground['leg_inbound'].map(mydict)
    return ground

In [118]:
ground = calcs_avg_catering_dur_dup_rows()
ground['duplicate_row'] = ground.duplicated(['day_of_origin', 'dep_leg_inbound', 'arr_leg_inbound', 'arr_leg_outbound', 'sched_inbound_dep',
                                             'sched_inbound_arr'], keep='first')
investigation = ground.loc[ground['duplicate_row'] == True]
ground.drop(investigation.index.to_list(), inplace=True)

In [119]:
# Rerun duplicate check.
ground = ground_finds_duplicates(ground, ['day_of_origin', 'ac_type', 'fn_number', 'ac_registration', 'dep_leg_inbound',
                                             'arr_leg_inbound', 'arr_leg_outbound', 'sched_inbound_dep', 'sched_inbound_arr'])
investigation = ground.loc[ground['duplicate_row'] == True]

investigation.shape #Check no further duplicates remain

(0, 20)

## Train-test split on Ground data

In [120]:
# Select the same days as in the flight train dataset for use in the Ground train dataset.
mydict = {}
for ac_reg in set(flight_train['ac_registration']):
    days = set(flight_train[flight_train['ac_registration'] == ac_reg]['day_of_origin'].to_list())
    mydict.update({ac_reg : days})

ground['train_test'] = 'test'
for k in mydict.keys():
    for date in mydict[k]:
        ground.loc[(ground['day_of_origin'] == date) & (ground['ac_registration'] == k), 'train_test'] = 'train'

In [121]:
# Run check that all ac_registrations in ground are in flight
ground_ac_reg = set(ground['ac_registration'].to_list())
flight_ac_reg = set(flight['ac_registration'].to_list())

# Elements in g_ac_reg but not in f_ac_reg
elements_in_g_not_in_f = ground_ac_reg - flight_ac_reg

# Elements in f_ac_reg but not in g_ac_reg
elements_in_f_not_in_g = flight_ac_reg - ground_ac_reg

print("Elements in ground_ac_reg but not in flight_ac_reg:", elements_in_g_not_in_f)
print("Elements in flight_ac_reg but not in ground_ac_reg:", elements_in_f_not_in_g)

Elements in ground_ac_reg but not in flight_ac_reg: set()
Elements in flight_ac_reg but not in ground_ac_reg: {'ECLGEX'}


In [122]:
ground_train = ground[ground['train_test'] == 'train']
ground_train.drop(columns=['train_test', 'duplicate_row'], inplace=True)
ground_test = ground[ground['train_test'] == 'test']
ground_test.drop(columns=['train_test', 'duplicate_row'], inplace=True)

In [123]:
# Saving the ground test files in their original format to use as pickle files for the imputation steps in the test
# dataset creation.

with open(pkl_path + 'ground_test.pkl', 'wb') as file:
    pickle.dump(ground_test, file)

with open(pkl_path + 'ground_train.pkl', 'wb') as file:
    pickle.dump(ground_train, file)

## Ground file: Sense check

### Sense checking: 'sched_outbound_dep' minus 'sched_inbound_arr' is the same as 'sched_turnaround'

In [124]:
# Convert dep_sched_date, arr_sched_date, m_offblockdt, m_onblockdt to datetime as they are currently object datatypes
to_datetime(['sched_outbound_dep', 'sched_inbound_arr', 'sched_inbound_dep', 'sched_outbound_arr', 'day_of_origin'], ground_train)

ground_train['taround_imputed'] = ((ground_train['sched_outbound_dep'].dt.hour - ground_train['sched_inbound_arr'].dt.hour)*60) + (
    ground_train['sched_outbound_dep'].dt.minute - ground_train['sched_inbound_arr'].dt.minute)

ground_train['taround_check'] = ground_train['taround_imputed'] - ground_train['sched_turnaround']
assert ground_train['taround_check'].all() == 0, "not equal to zero"

# sense check fine

### Sense-checking columns: 'day_of_origin' versus 'sched_inbound_dep_date', 'sched_inbound_arr_date', 'sched_outbound_dep_date', 'sched_outbound_arr_date'

In [125]:
# Checking that the 'day_of_origin' is the same as 'sched_inbound_dep_date', 'sched_inbound_arr_date', 'sched_outbound_dep_date',
# 'sched_outbound_arr_date'

for col in ['sched_inbound_dep', 'sched_inbound_arr', 'sched_outbound_dep', 'sched_outbound_arr']:
    ground_train[col + '_date_check'] = ground_train[col].dt.date == ground_train['day_of_origin'].dt.date

In [126]:
ground_train.loc[ground_train['sched_inbound_dep_date_check'] != True] #All on same day

in_arr_dat = ground_train.loc[ground_train['sched_inbound_arr_date_check'] != True]
ou_dep_dat = ground_train.loc[ground_train['sched_outbound_dep_date_check'] != True]
ou_arr_dat = ground_train.loc[ground_train['sched_outbound_arr_date_check'] != True]

investigation = pd.concat([in_arr_dat, ou_dep_dat, ou_arr_dat], ignore_index=False)
investigation = investigation.reset_index()
investigation = investigation.drop_duplicates(subset=["index"])
investigation = investigation[['day_of_origin', 'sched_inbound_dep', 'sched_inbound_arr', 'sched_outbound_dep', 'sched_outbound_arr']]

In [127]:
# Concatenate counts of the hours from columns above, fill missing values with 0
investigation_summary = pd.concat([
    investigation['sched_inbound_dep'].dt.hour.value_counts(),
    investigation['sched_inbound_arr'].dt.hour.value_counts(),
    investigation['sched_outbound_dep'].dt.hour.value_counts(),
    investigation['sched_outbound_arr'].dt.hour.value_counts()
], axis=1)

investigation_summary.columns = ['sched_inbound_dep', 'sched_inbound_arr', 'sched_outbound_dep', 'sched_outbound_arr']
investigation_summary = investigation_summary.fillna(0)

# Find missing hours
mylist = []
for i in range(min(investigation_summary.index), max(investigation_summary.index)):
    if i not in investigation_summary.index:
        mylist.append(i)

# Function to repeat list n times - used below.
def repeat_list_flat(input_list, n):
    return input_list * n

# Repeat 0s for each missing hour n times
sched_inbound_dep = repeat_list_flat([0], len(mylist))
sched_inbound_arr = repeat_list_flat([0], len(mylist))
sched_outbound_dep = repeat_list_flat([0], len(mylist))
sched_outbound_arr = repeat_list_flat([0], len(mylist))

mydict = {
    'sched_inbound_dep': sched_inbound_dep,
    'sched_inbound_arr': sched_inbound_arr,
    'sched_outbound_dep': sched_outbound_dep,
    'sched_outbound_arr': sched_outbound_arr
}

# Create a DataFrame from the dictionary
mydf = pd.DataFrame(data=mydict, index=mylist)

# Concatenate the missing hours DataFrame with original summary DataFrame
investigation_summary = pd.concat([investigation_summary, mydf], axis=0)
investigation_summary = investigation_summary.reset_index()
investigation_summary.rename(columns={'index': 'hour_departure'}, inplace=True)

In [128]:
# Visualisng the above
mylist = []
col_names = [col for col in investigation_summary if col != 'hour_departure']
for name in col_names:
    section = investigation_summary[['hour_departure', name]]
    x = section['hour_departure'].to_list()
    y = section[name].to_list()
    mylist.append(x)
    mylist.append(y)

fig = make_subplots(rows=2, cols=2, subplot_titles=col_names)
fig.add_trace(go.Bar(x=mylist[0], y=mylist[1],
                    marker=dict(color=mylist[0], colorscale="cividis")),
              1, 1)
fig.add_trace(go.Bar(x=mylist[2], y=mylist[3],
                    marker=dict(color=mylist[2], )),
              1, 2)
fig.add_trace(go.Bar(x=mylist[4], y=mylist[5],
                    marker=dict(color=mylist[4], colorscale="cividis")),
              2, 1)
fig.add_trace(go.Bar(x=mylist[6], y=mylist[7],
                    marker=dict(color=mylist[6], colorscale="cividis")),
              2, 2)

fig.update_layout(height=800, width=1100,
                  title_text='Count by hour where "Day of Origin" not the same as day of Inbound Arrival, Outbound Departure or Outbound Arrival', showlegend=False)
fig.update_xaxes(range=[3,24], title="Hour flight departed",)
fig.update_yaxes(title="Count",)
fig.update_coloraxes(showscale=False)
fig.show()

In [129]:
# Considering that these are arriving at early times in the morning it makes sense that it was an overnight flight

### Sense-checking columns: most frequently occuring value for 'mingt', for each 'ac_registration'

In [130]:
# For each aircraft registration, it makes sense that the mimimum ground_train time for an aircraft is specific to the aircraft
# itself, thus the most frequently occurring value should be the correct value to use.

# Group by 'ac_registration' and apply mode imputation
def mingt_mode_imputer(df):
    imp_mode_df = df.groupby('ac_registration')['mingt'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reset_index()
    mydict = {ac_reg: mgt for (ac_reg, mgt) in zip(imp_mode_df['ac_registration'], imp_mode_df['mingt'])}
    df['mingt_mode_imputed'] = df['ac_registration'].map(mydict)
    return df

ground_train = mingt_mode_imputer(ground_train)

ground_train = ground_train[['day_of_origin', 'ac_type', 'fn_number', 'ac_registration', 'mingt', 'dep_leg_inbound', 'arr_leg_inbound', 'arr_leg_outbound',
       'sched_inbound_dep', 'sched_inbound_arr', 'sched_outbound_dep', 'sched_outbound_arr', 'sched_turnaround', 'leg_inbound', 'leg_outbound',
       'catering_duration', 'cleaning_duration', 'pax_boarding_duration', 'taround_imputed', 'mingt_mode_imputed']]

ground_train.head(2)

,day_of_origin,ac_type,fn_number,ac_registration,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,taround_imputed,mingt_mode_imputed
0,2019-06-01,321,EC3292,ECLBAX,45,New Jessica,East Carmen,South Nathaniel,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 08:20:00,2019-06-01 10:35:00,95,272024970,272022230,27.0,NaN,25.0,95,45
1,2019-06-01,321,EC3258,ECLBAX,45,South Nathaniel,East Carmen,Joneshaven,2019-06-01 11:50:00,2019-06-01 14:15:00,2019-06-01 15:35:00,2019-06-01 17:00:00,80,272212848,271997824,25.0,NaN,NaN,80,45


### Sense-checking columns: 'sched_inbound_arr' is later than 'sched_inbound_dep'

In [131]:
# Function to check that one column is chronologically later than the other
def checks_col_later_than_other(df, col1, col2):
    mylist = []
    subset = df[[col1, col2]]
    for idx, row in subset.iterrows():
        i = pd.Timestamp(row[0]).timestamp()
        ii = pd.Timestamp(row[1]).timestamp()
        if ii < i:
            mylist.append(idx)
    if not len(mylist):
        print('All in ' + col2 + ' later than in ' + col1)
    return mylist

In [132]:
checks_col_later_than_other(ground_train, 'sched_inbound_dep', 'sched_inbound_arr')
# sense check fine

All in sched_inbound_arr later than in sched_inbound_dep


[]

### Sense-checking columns: 'sched_outbound_arr' is later than 'sched_outbound_dep'

In [133]:
checks_col_later_than_other(ground_train, 'sched_outbound_dep', 'sched_outbound_arr')
# sense check fine

All in sched_outbound_arr later than in sched_outbound_dep


[]

### Sense-checking columns: 'arr_sched_datetime' is later than 'dep_sched_datetime'

In [134]:
checks_col_later_than_other(flight_train, 'dep_sched_datetime', 'arr_sched_datetime')
# sense check fine

All in arr_sched_datetime later than in dep_sched_datetime


[]

### Sense-checking columns: 'm_onblockdt_imputed' is later than 'm_offblockdt'

In [135]:
checks_col_later_than_other(flight_train, 'm_offblockdt', 'm_onblockdt_imputed')
# sense check fine

All in m_onblockdt_imputed later than in m_offblockdt


[]

## Missing value analysis

In [136]:
null_printer(ground_train)

--------------------------------------------------------------------------------
                   Count & Percentage Missing Value by Column                   
--------------------------------------------------------------------------------
cleaning_duration             : 2175           :             99.90813045475426
pax_boarding_duration         : 200            :             9.186954524575103
arr_leg_outbound              : 3              :           0.13780431786862654
day_of_origin                 : 0              :                           0.0
sched_outbound_arr            : 0              :                           0.0
taround_imputed               : 0              :                           0.0
catering_duration             : 0              :                           0.0
leg_outbound                  : 0              :                           0.0
leg_inbound                   : 0              :                           0.0
sched_turnaround              : 0             

In [137]:
# Cleaning duration column will be completely dropped since there are only 3 rows with data.
ground_train.drop(['cleaning_duration'], axis=1, inplace=True)

### Calculating average value for blank pax_boarding_durations, for each 'ac_type' and destination legs

In [138]:
# Pax boarding duration should be dependent on the aircraft type and the airports (inbound and outbound).
# Since the aircraft flies to the arr_leg_inbound (arriving airport) and then redeparts from there to arr_leg_outbound, we can
# consider this as a departing airport as well.
# This is since we do not know what specific point of the trip the pax boarding duration refers to.
# Will need to be revisited once the files are combined to see if departure point it refers to can be deduced further.

In [139]:
# arr_leg_outbound blanks can be retrieved from the flight file.

In [140]:
# Since flight is a far better represented data file, use this to complete where blanks exist for 'arr_leg_outbound' in
# ground_train file.
ground_train = ground_train.sort_values(['ac_registration', 'day_of_origin', 'sched_inbound_dep'], ascending = [True, True, True]).reset_index(drop=True)
investigation = ground_train.loc[ground_train['arr_leg_outbound'].isna()]

mylist = []
for idx, row in investigation.iterrows():
    subset = flight.loc[(flight['dep_sched_datetime'].dt.date == row.day_of_origin.date()) & (flight['fn_number'] == row.fn_number) &
    (flight['dep_ap_sched'] == row.dep_leg_inbound) & (flight['arr_ap_sched'] == row.arr_leg_inbound) &
    (flight['ac_registration'] == row.ac_registration)]
    idx_to_match = subset.index[0] + 1
    mylist.append(idx_to_match)

matched = flight.iloc[mylist]['arr_ap_sched'].to_list()

for idx, matched_loc in zip(investigation.index, matched):
    ground_train.iloc[idx, ground_train.columns.get_loc('arr_leg_outbound')] = matched_loc

# check updated correctly
ground_train.iloc[investigation.index]

,day_of_origin,ac_type,fn_number,ac_registration,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,pax_boarding_duration,taround_imputed,mingt_mode_imputed
959,2019-06-04,319,EC2614,ECLDGX,40,Yoderburgh,East Carmen,North Jeffrey,2019-06-04 05:00:00,2019-06-04 06:35:00,2019-06-04 07:40:00,2019-06-04 08:25:00,65,271961157,272352650,26.0,NaN,65,40
1675,2019-06-12,E95,EC2836,ECLWGX,40,Davidtown,East Carmen,West Jason,2019-06-12 12:00:00,2019-06-12 13:55:00,2019-06-12 14:45:00,2019-06-12 16:00:00,50,272229269,271969616,22.0,20.0,50,40
1951,2019-06-22,E95,EC2836,ECLWPX,40,Davidtown,East Carmen,Port Bobby,2019-06-22 12:00:00,2019-06-22 13:55:00,2019-06-22 14:35:00,2019-06-22 15:40:00,40,272229279,272350803,22.0,NaN,40,40


In [141]:
# Now no further null values remain in the ground_train file.
# Further sense checking is required that the 'dep_leg_inbound' is not the same as 'arr_leg_inbound', is not the same
# as 'arr_leg_outbound'

ground_train['dep_in_equals_arr_in'] = ground_train['dep_leg_inbound'] == ground_train['arr_leg_inbound']
ground_train['arr_in_equals_arr_out'] = ground_train['arr_leg_inbound'] == ground_train['arr_leg_outbound']
investigation = ground_train.loc[(ground_train['dep_in_equals_arr_in'] == True) | (ground_train['arr_in_equals_arr_out'] == True)]

mylist = []
for idx, row in investigation.iterrows():
    subset = flight.loc[(flight['dep_sched_datetime'].dt.date == row.day_of_origin.date()) & (flight['fn_number'] == row.fn_number) &
    (flight['dep_ap_sched'] == row.dep_leg_inbound) & (flight['arr_ap_sched'] == row.arr_leg_inbound) &
    (flight['ac_registration'] == row.ac_registration) & (flight['dep_sched_datetime']== row.sched_inbound_dep)]
    idx_to_match = subset.index[0] + 1
    mylist.append(idx_to_match)

matched = flight.iloc[mylist]['arr_ap_sched'].to_list()

for idx, matched_loc in zip(investigation.index, matched):
    ground_train.iloc[idx, ground_train.columns.get_loc('arr_leg_outbound')] = matched_loc

# check updated correctly
ground_train.iloc[investigation.index]

,day_of_origin,ac_type,fn_number,ac_registration,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,pax_boarding_duration,taround_imputed,mingt_mode_imputed,dep_in_equals_arr_in,arr_in_equals_arr_out
1198,2019-06-22,DH4,EC3174,ECLGJX,35,Lake Kevin,East Carmen,Port Julieview,2019-06-22 05:00:00,2019-06-22 05:45:00,2019-06-22 06:55:00,2019-06-22 08:05:00,70,272011946,271995702,28.0,15.0,70,35,False,True
1330,2019-06-19,DHX,EC3230,ECLGOX,35,Haynesside,East Carmen,Port Courtneytown,2019-06-19 13:05:00,2019-06-19 14:20:00,2019-06-19 15:20:00,2019-06-19 16:05:00,60,272210730,272012153,23.0,17.0,60,35,False,True
1456,2019-06-06,E95,EC2788,ECLWAX,40,Castroville,East Carmen,East Melindachester,2019-06-06 04:45:00,2019-06-06 07:10:00,2019-06-06 08:25:00,2019-06-06 10:40:00,75,271980429,272007020,17.0,214.0,75,40,False,True
1653,2019-06-02,E95,EC2974,ECLWGX,40,West Danielport,East Carmen,Michaelfurt,2019-06-02 05:05:00,2019-06-02 06:35:00,2019-06-02 07:20:00,2019-06-02 09:10:00,45,271992571,272217507,16.0,17.0,45,40,False,True


In [142]:
ground_train.drop(['dep_in_equals_arr_in', 'arr_in_equals_arr_out'], axis=1, inplace=True)

## Combining ground file into 1 vertical file

In [143]:
# Combining ground_train into 1 vertical file (not horizontal). This is to assist the creation of the final superfile, since the data will have the
# same row-wise formatting.

ground_train_a = ground_train[['day_of_origin', 'ac_type', 'fn_number', 'ac_registration', 'dep_leg_inbound', 'arr_leg_inbound', 'sched_inbound_dep',
                   'sched_inbound_arr', 'leg_inbound', 'mingt_mode_imputed']]

ground_train_b = ground_train[['day_of_origin', 'ac_type', 'fn_number', 'ac_registration', 'arr_leg_inbound', 'arr_leg_outbound', 'sched_outbound_dep',
       'sched_outbound_arr', 'leg_outbound', 'mingt_mode_imputed']]

ground_train_a['leg_no'] = ground_train_a['leg_inbound']
ground_train_a.drop('leg_inbound', inplace=True, axis=1)
ground_train_b['leg_no'] = ground_train_b['leg_outbound']
ground_train_b.drop('leg_outbound', inplace=True, axis=1)

ground_train_a['dep_ap_sched'] = ground_train_a['dep_leg_inbound']
ground_train_a.drop('dep_leg_inbound', inplace=True, axis=1)
ground_train_b['dep_ap_sched'] = ground_train_b['arr_leg_inbound']
ground_train_b.drop('arr_leg_inbound', inplace=True, axis=1)

ground_train_a['arr_ap_sched'] = ground_train_a['arr_leg_inbound']
ground_train_a.drop('arr_leg_inbound', inplace=True, axis=1)
ground_train_b['arr_ap_sched'] = ground_train_b['arr_leg_outbound']
ground_train_b.drop('arr_leg_outbound', inplace=True, axis=1)

ground_train_a['dep_sched_datetime'] = ground_train_a['sched_inbound_dep']
ground_train_a.drop('sched_inbound_dep', inplace=True, axis=1)
ground_train_b['dep_sched_datetime'] = ground_train_b['sched_outbound_dep']
ground_train_b.drop('sched_outbound_dep', inplace=True, axis=1)

ground_train_a['arr_sched_datetime'] = ground_train_a['sched_inbound_arr']
ground_train_a.drop('sched_inbound_arr', inplace=True, axis=1)
ground_train_b['arr_sched_datetime'] = ground_train_b['sched_outbound_arr']
ground_train_b.drop('sched_outbound_arr', inplace=True, axis=1)

ground_train_a.rename(columns={'ac_type':'Ac_Type_Code'}, inplace=True)
ground_train_b.rename(columns={'ac_type':'Ac_Type_Code'}, inplace=True)

ground_train_a.columns == ground_train_b.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [144]:
# Check that the split has worked
ground_composite = pd.concat([ground_train_a, ground_train_b])

assert ground_composite.shape[0] == ground_train_a.shape[0] + ground_train_b.shape[0] , 'error'

In [145]:
# Recalculate lagged values
def calcs_duration_between_lines(df, col1, col2, name_new_col):
    mylist1 = []
    for reg in OrderedSet(df['ac_registration'].values):
        _mylist1 = []
        subset = df.loc[df['ac_registration'] == reg][[col1, col2]]
        subset_ord = subset.iloc[:-1].reset_index()
        subset_lag = subset.iloc[1:].reset_index()

        diff = (subset_lag[col1] - subset_ord[col2])/ np.timedelta64(1, 'm')
        _mylist1.extend(diff.to_list())
        _mylist1.extend([0])
        mylist1.append(_mylist1)

    mylist1 = [i for item in mylist1 for i in item]
    df[name_new_col] = mylist1
    return df

In [146]:
ground_composite = ground_composite[['day_of_origin', 'leg_no', 'fn_number', 'ac_registration', 'dep_ap_sched',
       'arr_ap_sched', 'dep_sched_datetime', 'arr_sched_datetime', 'Ac_Type_Code', 'mingt_mode_imputed']]
ground_composite = ground_composite.sort_values(['ac_registration', 'dep_sched_datetime'], ascending = [True, True]).reset_index(drop=True)
ground_composite = calcs_duration_between_lines(ground_composite, 'dep_sched_datetime', 'arr_sched_datetime', 'taround_imputed')
ground_composite.head(2)

,day_of_origin,leg_no,fn_number,ac_registration,dep_ap_sched,arr_ap_sched,dep_sched_datetime,arr_sched_datetime,Ac_Type_Code,mingt_mode_imputed,taround_imputed
0,2019-06-01,272024970,EC3292,ECLBAX,New Jessica,East Carmen,2019-06-01 03:25:00,2019-06-01 06:45:00,321,45,95.0
1,2019-06-01,272022230,EC3292,ECLBAX,East Carmen,South Nathaniel,2019-06-01 08:20:00,2019-06-01 10:35:00,321,45,75.0


In [147]:
# Check can use leg_no as unique identifier in ground_composite
investigation = ground_composite['leg_no'].unique()
assert investigation.shape[0] == ground_composite.shape[0], "error"

In [148]:
# Check can use leg_no as unique identifier in flight
investigation = flight_train['leg_no'].unique()
assert investigation.shape[0] == flight_train.shape[0], "error"

In [149]:
# Duplicates on fn_number obviously exist due to the combination of the 2 files vertically. Thus need to match with flight
# to establish which to remove and replace with correct fn_numbers, sourced from flight.
for leg in ground_composite['leg_no'].to_list():
    subset = flight_train.loc[flight_train['leg_no'] == leg]['fn_number']
    if not subset.empty:
        ground_composite.loc[ground_composite['leg_no'] == leg, ['fn_number_flight']] = subset.values[0]

In [150]:
# Non matches mean that these are flights which exist in ground and not in flight.
ground_composite.loc[(ground_composite['fn_number_flight'].isna()) & (ground_composite['fn_number'].notna())].head(2)

,day_of_origin,leg_no,fn_number,ac_registration,dep_ap_sched,arr_ap_sched,dep_sched_datetime,arr_sched_datetime,Ac_Type_Code,mingt_mode_imputed,taround_imputed,fn_number_flight
151,2019-06-12,272213781,EC2582,ECLBCX,East Carmen,East Latashaview,2019-06-12 18:15:00,2019-06-12 21:40:00,321,45,335.0,NaN
769,2019-06-14,271985390,EC3328,ECLBOX,East Carmen,Kennethfort,2019-06-15 05:05:00,2019-06-15 07:15:00,320,40,10130.0,NaN


In [151]:
ground_composite = ground_composite[['day_of_origin', 'leg_no', 'fn_number', 'fn_number_flight', 'ac_registration', 'dep_ap_sched',
                  'arr_ap_sched', 'dep_sched_datetime', 'arr_sched_datetime', 'Ac_Type_Code', 'mingt_mode_imputed', 'taround_imputed']]

### Investigating discrepancies in Ac_Type_code between the two files

In [152]:
mydf = pd.concat([ground_composite['Ac_Type_Code'].value_counts(), flight['Ac_Type_Code'].value_counts()], axis=1)
mydf.columns = ['ground', 'flight']
mydf = mydf.reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=mydf.index.to_list(),
    y=mydf['ground'].to_list(),
    name='Ground file',
    marker_color='darkturquoise'
))
fig.add_trace(go.Bar(
    x=mydf.index.to_list(),
    y=mydf['flight'].to_list(),
    name='Flight file',
    marker_color='rebeccapurple'
))
fig.update_layout(height=400, width=1000, title_text='Ac Type Code value by file name')
fig.show()

In [153]:
# This probably means that the fields 319, 321, 322 in Ground are 320 in Flight, will confirm.

investigation = ground_composite.loc[(ground_composite['Ac_Type_Code'] == '319') | (ground_composite['Ac_Type_Code'] == '321') |
                                                                                    (ground_composite['Ac_Type_Code'] == '322')]
investigation = investigation[investigation['fn_number_flight'].notnull()]

mylist = []
for idx, row in investigation.iterrows():
    lookup = row.leg_no
    flight_code = flight_train.loc[flight_train['leg_no'] == lookup]['Ac_Type_Code'].values[0]
    mylist.append((row.Ac_Type_Code, flight_code))

ground_code = [item[0] for item in mylist]
flight_code = [item[1] for item in mylist]

mydf1 = pd.DataFrame(ground_code, columns=["ground"])
mydf2 = pd.DataFrame(flight_code, columns=["flight"])
mydf = pd.concat([mydf1, mydf2], axis=1)
mydf['count'] = 1

summary = mydf.pivot_table(index=['ground', 'flight'], values='count', aggfunc='sum')
summary = summary.reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=summary['ground'].to_list(),
    y=summary['count'].to_list(),
    name='Ground file',
    marker_color='darkturquoise'
))
fig.add_trace(go.Bar(
    x=summary['flight'].to_list(),
    y=summary['count'].to_list(),
    name='Flight file',
    marker_color='rebeccapurple'
))
fig.update_layout(height=400, width=1000, title_text='Flight Ac Type Code for Ground Ac Type Code 319, 321, 322')
fig.show()

In [154]:
# Similarly DHX could be DH4, will confirm.

investigation = ground_composite.loc[ground_composite['Ac_Type_Code'] == 'DHX']
investigation = investigation[investigation['fn_number_flight'].notnull()]

mylist = []
for idx, row in investigation.iterrows():
    lookup = row.leg_no
    flight_code = flight.loc[flight['leg_no'] == lookup]['Ac_Type_Code'].values[0]
    mylist.append((row.Ac_Type_Code, flight_code))

ground_code = [item[0] for item in mylist]
flight_code = [item[1] for item in mylist]

mydf1 = pd.DataFrame(ground_code, columns=["ground"])
mydf2 = pd.DataFrame(flight_code, columns=["flight"])
mydf = pd.concat([mydf1, mydf2], axis=1)
mydf['count'] = 1

summary = mydf.pivot_table(index=['ground', 'flight'], values='count', aggfunc='sum')
summary = summary.reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=summary['ground'].to_list(),
    y=summary['count'].to_list(),
    name='Ground file',
    marker_color='darkturquoise'
))
fig.add_trace(go.Bar(
    x=summary['flight'].to_list(),
    y=summary['count'].to_list(),
    name='Flight file',
    marker_color='rebeccapurple'
))
fig.update_layout(height=400, width=1000, title_text='Flight Ac Type Code for Ground Ac Type Code DHX')
fig.show()

In [155]:
# Since DHX is DH4 and 319, 321 and 322 are 320, these will be replaced in the ground file.
ground_composite.loc[(ground_composite['Ac_Type_Code'] == '319') | (ground_composite['Ac_Type_Code'] == '321') |  (ground_composite['Ac_Type_Code'] == '322'),
['Ac_Type_Code']] = '320'
ground_composite.loc[ground_composite['Ac_Type_Code'] == 'DHX', ['Ac_Type_Code']] = 'DH4'

ground_composite['Ac_Type_Code'].value_counts()

Ac_Type_Code
320    2326
E95    1104
DH4     924
Name: count, dtype: int64

## Merging Ground and Flight into 1 Superfile

In [156]:
ground = ground_composite

In [157]:
# Check how the data and aircraft registrations compare across the two files.
num_in_flight_train_not_in_ground = 0
for leg in flight_train['leg_no'].to_list():
    subset = ground.loc[ground['leg_no'] == leg]['leg_no']
    if subset.empty:
        num_in_flight_train_not_in_ground += 1

num_in_ground_not_in_flight_train = 0
for leg in ground['leg_no'].to_list():
    subset = flight_train.loc[flight_train['leg_no'] == leg]['leg_no']
    if subset.empty:
        num_in_ground_not_in_flight_train += 1

print('Number in flight_train file, but not in ground:', num_in_flight_train_not_in_ground)
print('Number in ground file, but not in flight_train:', num_in_ground_not_in_flight_train)

Number in flight_train file, but not in ground: 2125
Number in ground file, but not in flight_train: 16


In [158]:
# Since we have far better data representation in the flight file, we can rather use a left join of flight with ground, to
# bring in the ming_mode_imputed field. As this was imputed with the mode for blank cells we can do this step again in
# the flight file.

merged = flight_train.merge(ground, how='left', left_on='leg_no', right_on='leg_no')
merged = merged[['day_of_origin_x', 'leg_no', 'fn_number_x', 'ac_registration_x', 'dep_ap_sched_x', 'arr_ap_sched_x', 'dep_sched_datetime_x',
                         'arr_sched_datetime_x', 'm_offblockdt', 'm_onblockdt', 'change_reason_code', 'dep_delay', 'arr_delay_imputed',
                 'arr_delay_imputed_null', 'Ac_Type_Code_x', 'trans_time', 'sched_trans_time', 'Crew_Group', 'Sched_Groundtime', 'Act_Groundtime',
                 'm_onblockdt_imputed', 'TLC_trans_pos', 'last_flight_indicator', 'first_flight_indicator', 'sched_duration', 'actual_duration',
                         'crew_type_change_imputed', 'Sched_Groundtime_imputed', 'Act_Groundtime_imputed',
                         'Crew_Group_imputed', 'change_reason_code_imputed', 'mingt_mode_imputed', 'taround_imputed', 'day_of_week']]

merged = merged.rename(columns={'day_of_origin_x': 'day_of_origin', 'fn_number_x' : 'fn_number', 'ac_registration_x' : 'ac_registration',
                       'dep_ap_sched_x' : 'dep_ap_sched', 'arr_ap_sched_x' : 'arr_ap_sched', 'dep_sched_datetime_x' : 'dep_sched_datetime',
                       'arr_sched_datetime_x' : 'arr_sched_datetime', 'Ac_Type_Code_x' : 'Ac_Type_Code'})
merged.head(1)

,day_of_origin,leg_no,fn_number,ac_registration,dep_ap_sched,arr_ap_sched,dep_sched_datetime,arr_sched_datetime,m_offblockdt,m_onblockdt,change_reason_code,dep_delay,arr_delay_imputed,arr_delay_imputed_null,Ac_Type_Code,trans_time,sched_trans_time,Crew_Group,Sched_Groundtime,Act_Groundtime,m_onblockdt_imputed,TLC_trans_pos,last_flight_indicator,first_flight_indicator,sched_duration,actual_duration,crew_type_change_imputed,Sched_Groundtime_imputed,Act_Groundtime_imputed,Crew_Group_imputed,change_reason_code_imputed,mingt_mode_imputed,taround_imputed,day_of_week
0,2019-06-01,272024970,EC3292,ECLBAX,New Jessica,East Carmen,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,other problem,25,16,0,320,0,0,Start,95.0,94.0,2019-06-01 07:01:00,"[Andrew Patterson_cp, Caleb Davidson_ca, Cassa...",0,1,200.0,191.0,"[ca, ca, ca, ca, cp, cp]",95,94,start,other problem,45.0,95.0,5


In [159]:
# 'taround_imputed' can be dropped since it is reliant on complete data in Ground, and the field is already calculated in Flight.
merged.drop(['taround_imputed'], axis=1, inplace=True)

# mingt_mode_imputed can be recalculated for empty cells - first rename before running function.
merged = merged.rename(columns={'mingt_mode_imputed' : 'mingt'})

In [160]:
investigation = merged[['ac_registration', 'mingt']]

ac_regs, ac_registration_mode = [], []
ac_registration = OrderedSet(investigation['ac_registration'].values)
for reg in ac_registration:
    subset = investigation.loc[investigation['ac_registration'] == reg]
    mode_array = subset['mingt'].mode().values
    if len(mode_array) == 1:
        ac_regs.append(reg)
        ac_registration_mode.append(mode_array[0])


for i, ii in zip(ac_regs, ac_registration_mode):
    merged.loc[merged['ac_registration'] == i, ['mingt_mode_imputed']] = ii

In [161]:
# Look for ac_registrations where the mode cannot be imputed
merged.loc[merged['mingt_mode_imputed'].isnull()]['ac_registration'].value_counts()

ac_registration
ECLGEX    5
Name: count, dtype: int64

In [162]:
# Find other characteristics of ECLGEX that it has in common with other ac_registrations, impute mingt_mode_imputed
# from this
merged.loc[merged['mingt_mode_imputed'].isnull()]['Ac_Type_Code'].value_counts()

Ac_Type_Code
DH4    5
Name: count, dtype: int64

In [163]:
# Thus will impute the mingt_mode from other 'Ac_Type_Code' = 'DH4'
dh4_mingt_mode_imputed = merged.loc[merged['Ac_Type_Code'] == 'DH4']['mingt_mode_imputed'].value_counts().index[0]

# Update the mingt_mode for ECLGEX to this value (35)
merged.loc[merged['ac_registration'] == 'ECLGEX', ['mingt_mode_imputed']] = dh4_mingt_mode_imputed
merged.drop(['mingt'], axis=1, inplace=True)

In [164]:
# Drop other columns where an imputed one removes the blanks
merged.drop(['Sched_Groundtime', 'm_onblockdt', 'Act_Groundtime'], axis=1, inplace=True)
null_printer(merged)

--------------------------------------------------------------------------------
                   Count & Percentage Missing Value by Column                   
--------------------------------------------------------------------------------
day_of_origin                 : 0              :                           0.0
leg_no                        : 0              :                           0.0
day_of_week                   : 0              :                           0.0
change_reason_code_imputed    : 0              :                           0.0
Crew_Group_imputed            : 0              :                           0.0
Act_Groundtime_imputed        : 0              :                           0.0
Sched_Groundtime_imputed      : 0              :                           0.0
crew_type_change_imputed      : 0              :                           0.0
actual_duration               : 0              :                           0.0
sched_duration                : 0             

In [165]:
# Export data for further analysis
data_path = '/content/drive/MyDrive/01_Data/02_Processed/'

merged.to_csv(data_path + 'merged.csv')